**상장 전처리 중)**

1. all 결측치인 컬럼 제거 O
2. 회계월이 12월이 아닌 row 제거 O
--------------산업 나누기------------------
1. 상폐가 아닌 기업의 상폐일은 모두 nan으로 설정 O
2. 외국계기업 제거 O
3. 거래소코드 2개인 기업 거래소코드 1개 제거 O
4. 연결개별합치기 -> 연결 비어있는 결측치들 개별값으로 채우기 O
5. 증가율 직접 구해서 inf 증가율 0으로 수정하기 O
--------------레이블링-----------------
1. SPAC 기업 제거하기 O
2. 상장일 이전 회계년도 데이터 제거 O
3. 총자본증가율, 자기자본증가율 수정 O
4. 결측치 및 이상치 확인 O
5. 단위 맞추기 O

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### 1. SPAC기업 제거하기
  - major(6784,120) -> major2(6572,120)
    - major 부실0: 6676, 부실1: 108
    - 제거 후, major2 부실0: 6465, 부실1: 107
  - minor(4860,130) -> minor2(4657,130)
    - minor 부실0: 4798, 부실1: 62
    - 제거 후, minor2 부실0: 4598, 부실1: 59



주력산업 major, 비주력산업 minor

##### SPAC기업 불러오기

In [ ]:
major = pd.read_csv('./drive/MyDrive/공유문서함/상장csv/240315상장_주력산업_레이블링.csv', index_col=0)
major

,회사명,거래소코드,회계년도,매출액총이익률,매출액정상영업이익률,매출액순이익률,총자본사업이익률,총자본정상영업이익률,총자본순이익률,자기자본정상영업이익률,자기자본순이익률,경영자본정상영업이익률,경영자본순이익률,자본금정상영업이익률,자본금순이익률,매출원가 대 매출액비율,영업비용 대 영업수익비율,기타손익비률,금융손익비율,금융비용부담률,외환이익 대 매출액비율,광고선전비 대 매출액비율,세금과공과 대 세금과공과 차감전순이익률,기업순이익률,수지비율(관계기업투자손익 제외),인건비 대 총비용비율,R & D 투자효율,세금과공과 대 총비용비율,금융비용 대 총비용비율,감가상각비 대 총비용비율,감가상각률,누적감가상각률,이자부담률,지급이자율,차입금평균이자율,유보율,적립금비율(재정비율),1주당매출액,1주당순이익,1주당 CASH FLOW,1주당순자산,1주당정상영업이익,유동자산구성비율,재고자산 대 유동자산비율,유동자산 대 비유동자산비율,당좌자산구성비율,비유동자산구성비율,자기자본구성비율,타인자본구성비율,자기자본배율,비유동비율,비유동장기적합률,유동비율,당좌비율,현금비율,매출채권비율,재고자산 대 순운전자본비율,매출채권 대 매입채무비율,매입채무 대 재고자산비율,부채비율,유동부채비율,단기차입금 대 총차입금비율,비유동부채비율,비유동부채 대 순운전자본비율,순운전자본비율,차입금의존도,차입금비율,유보액대비율,유보액 대 납입자본배율,유동자산집중도,비유동자산집중도,투자집중도,CASH FLOW 대 부채비율,CASH FLOW 대 차입금비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,정상영업이익대비이자보상배율,총자본회전률,경영자본회전률,자기자본회전률,자본금회전률,타인자본회전률,매입채무회전률,매입채무회전기간,유동자산회전률,당좌자산회전률,재고자산회전률,재고자산회전기간,재공품회전률,매출채권회전률,매출채권회전기간,비유동자산회전률,유형자산회전율,순운전자본회전률,운전자본회전률,1회전기간,부가가치(백만원),총자본투자효율,설비투자효율,기계투자효율,부가가치율,노동소득분배율,자본분배율,이윤분배율,설립일,상장일,상장폐지일,영업활동으로 인한 현금흐름(간접법),투자활동으로 인한 현금흐름,재무활동으로 인한 현금흐름,유동부채,자산,자본,매출액(수익),상장협 산업분류(소분류),상장협 산업분류(중분류),매출액증가율,유형자산증가율,비유동생물자산증가율,투자부동산증가율,유동자산증가율,비유동자산증가율,총자본증가율,자기자본증가율,재고자산증가율,정상영업이익증가율,순이익증가율,총포괄이익증가율,자산.1,label
0,제이앤유글로벌,86200,2014-12-01,13.73,-23.71,-30.69,-12.10,-12.67,-16.41,-20.52,-26.16,-13.04,-16.88,-219.19,-277.41,86.27,123.71,-6.73,-1.82,2.30,-2.23,0.98,0.00,-15.18,128.28,22.41,0.00,0.0,1.68,3.81,12.12,0.00,3.21,6.77,6.24,841.83,90.03,4623.03,-1407.0,-454.84,4236.75,-1095.93,62.96,21.51,169.98,49.42,37.04,59.75,40.25,922.83,61.99,58.84,169.95,133.39,47.20,72.58,52.26,240.78,30.15,67.35,62.00,97.69,5.36,12.35,25.91,29.27,48.99,54.59,8.57,127.47,59.58,161.96,-0.70,-0.97,-0.28,-0.48,-10.33,0.53,0.55,0.87,9.25,1.40,17.09,21.35,0.78,1.01,3.56,102.50,63.04,5.89,62.01,1.68,2.40,0.58,0.00,0.01,3635.27,4.48,18.40,168.74,7.60,401.64,-301.64,-403.56,1999/07/20,2006-06-07,2016-05-09,-4703654.0,1357391.0,-6014052.0,30068706.0,81164752.0,48498762.0,47807917.0,통신 및 방송 장비 제조업,"전자부품, 컴퓨터, 영상, 음향 및 통신장비 제조업",-0.273475,-0.007755,0.0,-0.026731,-0.277981,0.117592,-0.217270,-0.306934,-0.306934,0.000000,0.000000,0.000000,81164751.0,1.0
1,제이앤유글로벌,86200,2015-12-01,26.96,-33.53,-40.47,-15.36,-15.97,-19.28,-30.99,-38.75,-16.76,-20.22,-243.34,-298.11,73.04,133.53,-6.83,-2.07,1.74,1.74,0.69,0.00,-18.45,135.46,25.91,0.00,0.0,1.17,3.23,10.52,77.27,1.71,7.95,6.37,559.42,84.43,3833.41,-1527.0,-1646.59,3867.10,-1285.35,71.22,26.06,247.45,52.66,28.78,44.45,55.55,643.91,64.75,47.40,181.29,134.04,20.76,78.16,58.13,220.22,35.49,124.98,88.38,8.25,36.60,50.94,31.93,16.86,37.92,38.76,5.76,141.47,68.44,214.76,-21.65,-71.36,-12.03,-27.11,-19.30,0.48,0.50,0.92,7.26,0.98,8.78,41.59,0.71,0.93,2.93,124.41,51.93,3.86,94.56,1.46,2.15,0.45,0.00,0.01,2002.69,2.13,10.53,112.68,4.80,804.84,-704.84,-843.53,1999/07/20,2006-06-07,2016-05-09,-17932044.0,10635203.0,650010.0,36966126.0,94097710.0,41825394.0,41747399.0,통신 및 방송 장비 제조업,"전자부품, 컴퓨터, 영상, 음향 및 통신장비 제조업",-0.126768,-0.040803,0.0,-0.027465,0.311429,-0.099171,-0.137599,0.589111,0.589111,0.000000,0.000000,0.000000,94097710.0,1.0
2,미래오토스,176440,2017-12-01,8.33,0.83,-0.99,1.81,1.68,-2.02,3.22,-3.88,1.70,-2.04,32.49,-39.10,91.67,99.17,-1.76,0.07,0.33,-1.44,0.00,0.00,-1.36,100.90,9.27,0.00,0.0,0.31,2.07,12.36,52.48,1.39,2.60,3.37,639.09,89.88,14741.70,-152.0,-434.18,3549.41,121.82,57.67,22.95,136.25,44.44,42.33,47.88,52.12,714.73,88.41,76.38,129.37,99.68,46.90,121.80,101.09,81.13,150.12,108.87,93.12,75.97,15.76,57.62,13.09,29.37,61.34,42.81,6.39,28.70,20.43,50.23,2.52,4.47,1.31,0.66,2.53,2.04,2.05,3.90,39.31,4.26,10.22,35.73,3.48,4.47,15.78,23.13,64.38,14.48,25.21,4.89,6.57,2.05,0.05,0.08,24257.86,23.45,73.22,93.15,11.87,81.96,18.04,-8.38,2008/10/07,2016-11-28,NaN,-6017826.0,-6080460.0,11297144.0,46122480.0,103459053.0,49531567.0,204322026.0,전자부품 제조업,"전자부품, 컴퓨터, 영상, 음향 및 통신장비 제조업",0.014681,0.162763,0.0,0.000000,0.035968,0.103204,-0.103356,0.122616,0.122616,-0.821602,-1.286612,-1.552762,103459053.0,0.0
3,미래오토스,176440,2018-12-01,10.68,3.55,0.10,7.

In [ ]:
major.shape

(6784, 130)

In [ ]:
# column별 결측치 개수 확인
nan_table = pd.DataFrame()

def find_nan(df):
  for column in df.columns:
    if df[column].isna().sum() != 0:
      print(column, df[column].isna().sum())
      # x = df[df[column].isna()]['회사명'].values
      # print(x)

  return nan_table

In [ ]:
find_nan(major)

1주당순자산 6776
상장폐지일 6680
자기자본증가율 176


""


In [ ]:
major.columns.tolist()

['회사명',
 '거래소코드',
 '회계년도',
 '매출액총이익률',
 '매출액정상영업이익률',
 '매출액순이익률',
 '총자본사업이익률',
 '총자본정상영업이익률',
 '총자본순이익률',
 '자기자본정상영업이익률',
 '자기자본순이익률',
 '경영자본정상영업이익률',
 '경영자본순이익률',
 '자본금정상영업이익률',
 '자본금순이익률',
 '매출원가 대 매출액비율',
 '영업비용 대 영업수익비율',
 '기타손익비률',
 '금융손익비율',
 '금융비용부담률',
 '외환이익 대 매출액비율',
 '광고선전비 대 매출액비율',
 '세금과공과 대 세금과공과 차감전순이익률',
 '기업순이익률',
 '수지비율(관계기업투자손익 제외)',
 '인건비 대 총비용비율',
 'R & D 투자효율',
 '세금과공과 대 총비용비율',
 '금융비용 대 총비용비율',
 '감가상각비 대 총비용비율',
 '감가상각률',
 '누적감가상각률',
 '이자부담률',
 '지급이자율',
 '차입금평균이자율',
 '유보율',
 '적립금비율(재정비율)',
 '1주당매출액',
 '1주당순이익',
 '1주당 CASH FLOW',
 '1주당순자산',
 '1주당정상영업이익',
 '유동자산구성비율',
 '재고자산 대 유동자산비율',
 '유동자산 대 비유동자산비율',
 '당좌자산구성비율',
 '비유동자산구성비율',
 '자기자본구성비율',
 '타인자본구성비율',
 '자기자본배율',
 '비유동비율',
 '비유동장기적합률',
 '유동비율',
 '당좌비율',
 '현금비율',
 '매출채권비율',
 '재고자산 대 순운전자본비율',
 '매출채권 대 매입채무비율',
 '매입채무 대 재고자산비율',
 '부채비율',
 '유동부채비율',
 '단기차입금 대 총차입금비율',
 '비유동부채비율',
 '비유동부채 대 순운전자본비율',
 '순운전자본비율',
 '차입금의존도',
 '차입금비율',
 '유보액대비율',
 '유보액 대 납입자본배율',
 '유동자산집중도',
 '비유동자산집중도',
 '투자집중도',
 'CASH FLOW 대 부채비율',


In [ ]:
major.rename(columns={'label':'부실'}, inplace=True)

In [ ]:
spac = pd.read_csv('./drive/MyDrive/공유문서함/상장_목적사업.csv', encoding='cp949')
spac.head()

,회사명,거래소코드,회계년도,목적사업
0,(주)CMG제약,58820,2010/12,16. 부동산업 및 임대업
1,(주)CMG제약,58820,2010/12,15. 전자부품 제조 및 판매업
2,(주)CMG제약,58820,2010/12,"18. 생물학적제제 개발, 제조 및 유통업"
3,(주)CMG제약,58820,2010/12,"19. 유전공학제제 개발, 제조 및 유통업"
4,(주)CMG제약,58820,2010/12,"20. 의약부외품의 개발, 제조 및 판매업"


In [ ]:
spac['목적사업'].isnull().sum()

0

In [ ]:
# 목적사업 존재하지 않는 row제거
spac = spac.dropna(subset=['목적사업'])

In [ ]:
spac

,회사명,거래소코드,회계년도,목적사업
0,(주)CMG제약,58820,2010/12,16. 부동산업 및 임대업
1,(주)CMG제약,58820,2010/12,15. 전자부품 제조 및 판매업
2,(주)CMG제약,58820,2010/12,"18. 생물학적제제 개발, 제조 및 유통업"
3,(주)CMG제약,58820,2010/12,"19. 유전공학제제 개발, 제조 및 유통업"
4,(주)CMG제약,58820,2010/12,"20. 의약부외품의 개발, 제조 및 판매업"
...,...,...,...,...
361773,흥아해운(주),3280,2019/12,14. 종합물류업
361774,흥아해운(주),3280,2019/12,"15. 부동산개발업, 해외부동산개발사업"
361775,흥아해운(주),3280,2019/12,"16. 복합레저업, 체육시설업, 관광숙박업"
361776,흥아해운(주),3280,2019/12,"17. 원유, 천연가스 및 기타석유제품의 개발, 수입, 저장, 판매사업"


In [ ]:
spac[spac['목적사업'].str.contains('합병')]['회사명'].unique()

array(['(주)고려시멘트', '(주)골프존', '(주)골프존뉴딘홀딩스', '(주)광림', '(주)국전약품', '(주)나노',
       '(주)나인테크', '(주)네온테크', '(주)넥슨게임즈', '(주)넵튠', '(주)누보', '(주)뉴온',
       '(주)다보링크', '(주)더블유에스아이', '(주)덴티스', '(주)드림시큐리티', '(주)디딤이앤에프',
       '(주)디비아이엔씨', '(주)디알텍', '(주)디와이디', '(주)러셀', '(주)레이크머티리얼즈',
       '(주)로보로보', '(주)맘스터치앤컴퍼니', '(주)모비스', '(주)모코엠시스', '(주)미래생명자원',
       '(주)바이오로그디바이스', '(주)밸로프', '(주)베노티앤알', '(주)본느', '(주)블루젬디앤씨',
       '(주)비비안', '(주)비올', '(주)비츠로시스', '(주)비투엔', '(주)삼기', '(주)상지카일룸',
       '(주)서진오토모티브', '(주)세림비앤지', '(주)세토피아', '(주)세화피앤씨', '(주)셀바스헬스케어',
       '(주)스튜디오산타클로스엔터테인먼트', '(주)스피어파워', '(주)썬테크놀로지스', '(주)썸에이지',
       '(주)쎄노텍', '(주)씨모텍', '(주)아이비김영', '(주)아이엘사이언스', '(주)아이톡시', '(주)알로이스',
       '(주)알톤스포츠', '(주)애니플러스', '(주)액션스퀘어', '(주)에스엠벡셀', '(주)에스와이코퍼레이션',
       '(주)에이루트', '(주)에이치앤비디자인', '(주)에이프로젠', '(주)에이프로테크놀로지', '(주)에치에프알',
       '(주)에코바이브', '(주)에프에스엔', '(주)엑셈', '(주)엔투텍', '(주)엔피', '(주)예선테크',
       '(주)오하임앤컴퍼니', '(주)옵투스제약', '(주)와이즈버즈', '(주)우정바이오', '(주)원바이오젠',
       '(주)웨이버스', '(주)위메이드플레

In [ ]:
# SPAC 기업의 회사명 리스트
spac_company_list = ['(주)고려시멘트', '(주)골프존', '(주)골프존뉴딘홀딩스', '(주)광림', '(주)국전약품', '(주)나노',
       '(주)나인테크', '(주)네온테크', '(주)넥슨게임즈', '(주)넵튠', '(주)누보', '(주)뉴온',
       '(주)다보링크', '(주)더블유에스아이', '(주)덴티스', '(주)드림시큐리티', '(주)디딤이앤에프',
       '(주)디비아이엔씨', '(주)디알텍', '(주)디와이디', '(주)러셀', '(주)레이크머티리얼즈',
       '(주)로보로보', '(주)맘스터치앤컴퍼니', '(주)모비스', '(주)모코엠시스', '(주)미래생명자원',
       '(주)바이오로그디바이스', '(주)밸로프', '(주)베노티앤알', '(주)본느', '(주)블루젬디앤씨',
       '(주)비비안', '(주)비올', '(주)비츠로시스', '(주)비투엔', '(주)삼기', '(주)상지카일룸',
       '(주)서진오토모티브', '(주)세림비앤지', '(주)세토피아', '(주)세화피앤씨', '(주)셀바스헬스케어',
       '(주)스튜디오산타클로스엔터테인먼트', '(주)스피어파워', '(주)썬테크놀로지스', '(주)썸에이지',
       '(주)쎄노텍', '(주)씨모텍', '(주)아이비김영', '(주)아이엘사이언스', '(주)아이톡시', '(주)알로이스',
       '(주)알톤스포츠', '(주)애니플러스', '(주)액션스퀘어', '(주)에스엠벡셀', '(주)에스와이코퍼레이션',
       '(주)에이루트', '(주)에이치앤비디자인', '(주)에이프로젠', '(주)에이프로테크놀로지', '(주)에치에프알',
       '(주)에코바이브', '(주)에프에스엔', '(주)엑셈', '(주)엔투텍', '(주)엔피', '(주)엔피디',
       '(주)예선테크', '(주)오하임앤컴퍼니', '(주)옵투스제약', '(주)와이즈버즈', '(주)우정바이오',
       '(주)원바이오젠', '(주)웨이버스', '(주)위메이드플레이', '(주)윙스풋', '(주)이노인스트루먼트',
       '(주)이랜시스', '(주)인산가', '(주)인스코비', '(주)일승', '(주)자비스', '(주)정다운',
       '(주)제이시스메디칼', '(주)지니틱스', '(주)지란지교시큐리티', '(주)카이노스메드', '(주)케이프',
       '(주)켐온', '(주)켐트로스', '(주)코원플레이', '(주)코퍼스코리아', '(주)큐브엔터테인먼트',
       '(주)클래시스', '(주)태성', '(주)토박스코리아', '(주)티에스넥스젠', '(주)티에스트릴리온',
       '(주)팬스타엔터프라이즈', '(주)포인트엔지니어링', '(주)포커스에이치엔에스', '(주)폴라리스세원',
       '(주)폴라리스오피스', '(주)프로이천', '(주)플레이그램', '(주)하이비젼시스템', '(주)한국비엔씨',
       '(주)한송네오텍', '(주)화신정공', '(주)휴센텍', 'KD(주)', '글로벌텍스프리(주)', '글로본(주)',
       '글로스텍(주)', '나무기술(주)', '나이스평가정보(주)', '대성홀딩스(주)', '디와이씨(주)',
       '마이크로로봇(주)', '미스터블루(주)', '바디텍메드(주)', '범양건영(주)', '삼성물산(주)',
       '소니드(주)', '소프트캠프(주)', '솔트웨어(주)', '씨아이에스(주)', '아이이(주)', '알서포트(주)',
       '알에프에이치아이씨(주)', '에스비더블유생명과학(주)', '에스지에이솔루션즈(주)', '에이아이비트(주)',
       '에이치엘디앤아이한라(주)', '에이치엘비글로벌(주)', '에코바이오홀딩스(주)', '엠에프엠코리아(주)',
       '와이아이케이(주)', '원텍(주)', '윈텍(주)', '줌인터넷(주)', '지엘팜텍(주)', '지오릿에너지(주)',
       '참존글로벌(주)', '케이씨그린홀딩스(주)', '케이에이치건설(주)', '케이엠제약(주)', '코리아에프티(주)',
       '패션플랫폼(주)', '한성크린텍(주)', '현대무벡스(주)', '현대코퍼레이션(주)', '휴림로봇(주)',
       '휴마시스(주)']

In [ ]:
len(spac_company_list)

156

In [ ]:
# (주) 빼기
spac_company_list = [company.replace('(주)', '') for company in spac_company_list]

##### 주력산업에서 SPAC기업 제거하기

In [ ]:
# major2는 spac기업 제거한 데이터프레임

In [ ]:
# '회사명' 컬럼의 값이 리스트에 포함되어 있는 행을 삭제
major2 = major[~major['회사명'].isin(spac_company_list)]

In [ ]:
display(major.shape)
display(major2.shape)

(6784, 130)

(6572, 130)

In [ ]:
display(major['부실'].value_counts())
display(major2['부실'].value_counts())

0.0    6676
1.0     108
Name: 부실, dtype: int64

0.0    6465
1.0     107
Name: 부실, dtype: int64

##### 비주력산업에서 SPAC기업 제거하기

In [ ]:
minor = pd.read_csv('./drive/MyDrive/공유문서함/상장csv/240315상장_비주력산업_레이블링.csv', index_col =0)
minor

,회사명,거래소코드,회계년도,매출액총이익률,매출액정상영업이익률,매출액순이익률,총자본사업이익률,총자본정상영업이익률,총자본순이익률,자기자본정상영업이익률,자기자본순이익률,경영자본정상영업이익률,경영자본순이익률,자본금정상영업이익률,자본금순이익률,매출원가 대 매출액비율,영업비용 대 영업수익비율,기타손익비률,금융손익비율,금융비용부담률,외환이익 대 매출액비율,광고선전비 대 매출액비율,세금과공과 대 세금과공과 차감전순이익률,기업순이익률,수지비율(관계기업투자손익 제외),인건비 대 총비용비율,R & D 투자효율,세금과공과 대 총비용비율,금융비용 대 총비용비율,감가상각비 대 총비용비율,감가상각률,누적감가상각률,이자부담률,지급이자율,차입금평균이자율,유보율,적립금비율(재정비율),1주당매출액,1주당순이익,1주당 CASH FLOW,1주당순자산,1주당정상영업이익,유동자산구성비율,재고자산 대 유동자산비율,유동자산 대 비유동자산비율,당좌자산구성비율,비유동자산구성비율,자기자본구성비율,타인자본구성비율,자기자본배율,비유동비율,비유동장기적합률,유동비율,당좌비율,현금비율,매출채권비율,재고자산 대 순운전자본비율,매출채권 대 매입채무비율,매입채무 대 재고자산비율,부채비율,유동부채비율,단기차입금 대 총차입금비율,비유동부채비율,비유동부채 대 순운전자본비율,순운전자본비율,차입금의존도,차입금비율,유보액대비율,유보액 대 납입자본배율,유동자산집중도,비유동자산집중도,투자집중도,CASH FLOW 대 부채비율,CASH FLOW 대 차입금비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,정상영업이익대비이자보상배율,총자본회전률,경영자본회전률,자기자본회전률,자본금회전률,타인자본회전률,매입채무회전률,매입채무회전기간,유동자산회전률,당좌자산회전률,재고자산회전률,재고자산회전기간,재공품회전률,매출채권회전률,매출채권회전기간,비유동자산회전률,유형자산회전율,순운전자본회전률,운전자본회전률,1회전기간,부가가치(백만원),총자본투자효율,설비투자효율,기계투자효율,부가가치율,노동소득분배율,자본분배율,이윤분배율,설립일,상장일,상장폐지일,영업활동으로 인한 현금흐름(간접법),투자활동으로 인한 현금흐름,재무활동으로 인한 현금흐름,유동부채,자산,자본,매출액(수익),상장협 산업분류(소분류),상장협 산업분류(중분류),매출액증가율,유형자산증가율,비유동생물자산증가율,투자부동산증가율,유동자산증가율,비유동자산증가율,자기자본증가율,재고자산증가율,정상영업이익증가율,순이익증가율,총포괄이익증가율,총자본증가율,자산.1,label
0,HJ 매그놀리아 용평 호텔 앤 리조트,70960,2017-12-01,22.57,15.98,8.09,3.77,3.73,1.89,8.33,4.21,3.76,1.90,13.92,7.04,77.43,84.02,-5.53,-5.12,5.12,-0.04,0.86,37.47,3.08,89.61,11.95,0.00,2.76,5.69,7.86,5.49,78.63,2.16,7.86,8.22,69.59,41.04,4356.18,352.0,370.00,8146.08,695.92,16.43,9.85,19.65,14.81,83.57,47.02,52.98,169.59,177.75,118.80,55.39,49.94,12.62,631.34,0.0,5053.47,12.49,112.69,63.07,85.26,49.62,0.0,-13.23,16.35,34.78,19.30,0.70,90.51,338.03,412.40,-0.13,-0.42,-0.07,-0.28,3.12,0.23,0.24,0.52,0.87,0.42,112.74,3.24,1.10,1.19,14.85,24.59,0.0,1.84,198.51,0.30,0.30,0.44,0.00,0.00,77704.69,8.95,10.95,0.000000e+00,37.06,29.06,70.94,21.82,2000/02/07,2016-05-27,NaN,17809537.0,-61501445.0,1684160.0,257429706.0,8.681565e+08,4.081812e+08,2.096774e+08,일반 및 생활 숙박시설 운영업,숙박업,-0.004863,0.039747,0.0,0.000000,-0.398240,0.048530,0.00000,-0.011091,0.102818,0.183780,0.182663,0.029611,8.681565e+08,0.0
1,HJ 매그놀리아 용평 호텔 앤 리조트,70960,2018-12-01,22.91,13.54,5.62,2.97,2.92,1.21,5.97,2.48,2.93,1.22,10.21,4.23,77.09,86.46,-6.32,-6.54,5.59,0.29,0.62,46.60,2.42,92.82,15.16,0.00,3.53,5.98,9.85,5.91,85.50,2.36,6.80,7.08,72.09,41.67,3768.46,212.0,1090.82,8287.53,510.31,12.80,12.88,14.67,11.15,87.20,50.96,49.04,172.09,171.13,113.29,55.57,48.41,31.88,161.78,0.0,1474.81,10.97,96.24,45.19,75.68,51.06,0.0,-10.23,18.97,37.23,21.35,0.72,67.97,395.38,446.72,8.82,22.80,4.33,19.38,2.42,0.22,0.22,0.44,0.75,0.42,112.52,3.24,1.47,1.66,13.22,27.61,0.0,3.29,111.02,0.25,0.26,0.52,0.00,0.01,74163.43,9.12,10.70,0.000000e+00,40.89,34.64,65.36,13.73,2000/02/07,2016-05-27,NaN,52504906.0,-17193893.0,-8132498.0,187144500.0,8.127759e+08,4.141706e+08,1.813886e+08,일반 및 생활 숙박시설 운영업,숙박업,-0.134916,-0.026166,0.0,0.000000,-0.270702,-0.023125,0.00000,-0.046241,-0.266717,-0.399175,-0.356503,0.014673,8.127759e+08,0.0
2,HJ 매그놀리아 용평 호텔 앤 리조트,70960,2019-12-01,10.26,-3.19,-10.66,-0.41,-0.50,-1.66,-1.02,-3.41,-0.50,-1.67,-1.71,-5.73,89.74,103.19,-10.31,-8.24,9.57,-0.91,0.77,0.00,-0.17,113.18,17.47,0.00,3.48,8.26,11.80,6.10,82.36,2.91,7.24,7.81,64.45,39.13,2689.02,-287.0,490.32,7979.61,-85.68,12.80,14.66,14.68,10.92,87.20,46.57,53.43,164.45,187.26,118.27,48.72,41.58,22.05,131.24,0.0,1205.05,10.89,114.75,56.42,64.88,58.33,0.0,-13.47,21.26,45.66,18.25,0.64,82.20,560.11,650.91,8.38,21.06,4.48,29.40,-0.33,0.16,0.16,0.32,0.54,0.30,80.72,4.52,1.22,1.41,8.82,41.38,0.0,6.08,60.08,0.18,0.18,0.42,0.00,0.01,45500.21,5.35,6.32,0.000000e+00,35.15,57.59,42.41,-30.31,2000/02/07,2016-05-27,NaN,23600746.0,-20538858.0,-13495542.0,223287000.0,8.499356e+08,3.957823e+08,1.294313e+08,일반 및 생활 숙박시설 운영업,숙박업,-0.286442,0.048206,0.0,0.000000,0.046091,0.045665,0.19096,0.190960,-1.167902,-2.353979,-2.409036,-0.044398,8.499356e+08,0.0
3,한국지역난방공사,71320,2011-12-01,5.54,3.48,-0.36,2.01,1.69,-0.18,5.34,0.23,2.04,-0.21,128.40,5.42,94.46,97.39,-3.75,-3.50,3.5

In [ ]:
# '회사명' 컬럼의 값이 리스트에 포함되어 있는 행을 삭제
minor2 = minor[~minor['회사명'].isin(spac_company_list)]

In [ ]:
minor.rename(columns={'label':'부실'}, inplace=True)

In [ ]:
display(minor.shape)
display(minor2.shape)

(4860, 130)

(4657, 130)

In [ ]:
display(minor['부실'].value_counts())
display(minor2['부실'].value_counts())

0.0    4798
1.0      62
Name: 부실, dtype: int64

0.0    4598
1.0      59
Name: 부실, dtype: int64

#### 2. 상장일 이전 회계년도 데이터 제거

##### 주력산업

In [ ]:
major2[['회계년도']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6572 entries, 0 to 6774
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   회계년도    6572 non-null   datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 360.7 KB


In [ ]:
major2[['상장일']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6572 entries, 0 to 6774
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   상장일     6572 non-null   datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 360.7 KB


In [ ]:
major2['상장일'] = pd.to_datetime(major2['상장일'])

In [ ]:
a = major2[major2['회계년도'] < major2['상장일']]

In [ ]:
a.shape

(21, 130)

In [ ]:
major2.shape

(6540, 130)

In [ ]:
de = pd.read_csv('./drive/MyDrive/공유문서함/상장_부실사유.csv', encoding='cp949')
de

,번호,회사명,종목코드,폐지일자,폐지사유,비고
0,47,파티게임즈,194510,2020-09-09,감사의견거절(감사범위 제한),NaN
1,45,코썬바이오,204990,2020-08-11,기업의 계속성 및 경영의 투명성 등을 종합적으로 고려하여 상장폐지기준에 해당한다고 결정,NaN
2,43,바이오빌,65940,2020-07-31,감사의견 거절(감사범위 제한 및 계속기업 존속능력에 대한 불확실성),NaN
3,35,모다,149940,2020-07-10,감사의견거절(감사범위 제한),NaN
4,33,화진,134780,2020-07-02,감사의견거절(감사범위제한),NaN
...,...,...,...,...,...,...
300,10,티이씨,67950,2010-02-20,당해 코스닥시장상장법인의 계속성 및 경영의 투명성 등을 종합적으로 고려한 결과 상장...,NaN
301,9,이롬텍,45400,2010-02-11,"정기보고서 미제출로 관리종목 지정후 법정제출기한내 사업보고서 미제출, 2회 연속 정...",NaN
302,8,글로포스트,37830,2010-02-09,"기업의 계속성, 경영의 투명성 또는 기타 코스닥시장의 건전성 등을 종합적으로 고려하...",NaN
303,7,디보스,80140,2010-02-02,자본잠식률 50%이상사유로 관리종목지정후 반기 감사의견 부적정,NaN


In [ ]:
de[de['회사명'] == '칩스앤미디어']

,번호,회사명,종목코드,폐지일자,폐지사유,비고


In [ ]:
a[['회사명','거래소코드','회계년도','상장일','부실']]

,회사명,거래소코드,회계년도,상장일,부실
57,오상헬스케어,36220,2013-12-01,2024-03-13,0.0
58,오상헬스케어,36220,2014-12-01,2024-03-13,1.0
59,오상헬스케어,36220,2015-12-01,2024-03-13,1.0
1737,엠씨넥스,97520,2012-12-01,2021-07-06,0.0
1747,엠씨넥스,97520,2013-12-01,2021-07-06,0.0
1748,엠씨넥스,97520,2013-12-01,2021-07-06,0.0
1751,엠씨넥스,97520,2014-12-01,2021-07-06,0.0
1752,엠씨넥스,97520,2014-12-01,2021-07-06,0.0
1759,엠씨넥스,97520,2015-12-01,2021-07-06,0.0
1760,엠씨넥스,97520,2015-12-01,2021-07-06,0.0


In [ ]:
major2.drop_duplicates(inplace=True)

In [ ]:
# 엠씨넥스: 한국거래소 유가증권시장 이전상장에 따른 코스닥시장 상장폐지 -> 제거
# 오상헬스케어: 2016년에 상폐되었다가 24년도 다시 상장됨 (당시에는 부실이었으니까 제거하지 않음)
# 칩스앤미디어 제거
# 비플라이소프트 제거

In [ ]:
major2.reset_index(drop=True, inplace=True)

In [ ]:
# 상장일 이전 회계년도 데이터가 있는 엠씨넥스, 칩스앤미디어, 비플라이소프트 제거
drop_index = major2[major2['회사명']== '엠씨넥스'].index
drop_index

Int64Index([1674, 1675, 1676, 1677, 1678, 1679, 1680, 1681, 1682, 1683, 1684,
            1685, 1686, 1687, 1688, 1689, 1690, 1691, 1692, 1693, 1694, 1695,
            1696, 1697, 1698, 1699, 1700, 1701, 1702, 1703],
           dtype='int64')

In [ ]:
len(drop_index)

30

In [ ]:
copy = major2.copy()

In [ ]:
copy.drop(drop_index, axis=0, inplace=True) # 엠씨넥스 제거

In [ ]:
copy.drop([2286], axis=0, inplace=True) # 칩스앤미디어 제거

In [ ]:
copy.drop([6214,6215], axis=0, inplace=True) # 비플라이소프트 제거

In [ ]:
copy[copy['회사명']== '비플라이소프트']

,회사명,거래소코드,회계년도,매출액총이익률,매출액정상영업이익률,매출액순이익률,총자본사업이익률,총자본정상영업이익률,총자본순이익률,자기자본정상영업이익률,자기자본순이익률,경영자본정상영업이익률,경영자본순이익률,자본금정상영업이익률,자본금순이익률,매출원가 대 매출액비율,영업비용 대 영업수익비율,기타손익비률,금융손익비율,금융비용부담률,외환이익 대 매출액비율,광고선전비 대 매출액비율,세금과공과 대 세금과공과 차감전순이익률,기업순이익률,수지비율(관계기업투자손익 제외),인건비 대 총비용비율,R & D 투자효율,세금과공과 대 총비용비율,금융비용 대 총비용비율,감가상각비 대 총비용비율,감가상각률,누적감가상각률,이자부담률,지급이자율,차입금평균이자율,유보율,적립금비율(재정비율),1주당매출액,1주당순이익,1주당 CASH FLOW,1주당순자산,1주당정상영업이익,유동자산구성비율,재고자산 대 유동자산비율,유동자산 대 비유동자산비율,당좌자산구성비율,비유동자산구성비율,자기자본구성비율,타인자본구성비율,자기자본배율,비유동비율,비유동장기적합률,유동비율,당좌비율,현금비율,매출채권비율,재고자산 대 순운전자본비율,매출채권 대 매입채무비율,매입채무 대 재고자산비율,부채비율,유동부채비율,단기차입금 대 총차입금비율,비유동부채비율,비유동부채 대 순운전자본비율,순운전자본비율,차입금의존도,차입금비율,유보액대비율,유보액 대 납입자본배율,유동자산집중도,비유동자산집중도,투자집중도,CASH FLOW 대 부채비율,CASH FLOW 대 차입금비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,정상영업이익대비이자보상배율,총자본회전률,경영자본회전률,자기자본회전률,자본금회전률,타인자본회전률,매입채무회전률,매입채무회전기간,유동자산회전률,당좌자산회전률,재고자산회전률,재고자산회전기간,재공품회전률,매출채권회전률,매출채권회전기간,비유동자산회전률,유형자산회전율,순운전자본회전률,운전자본회전률,1회전기간,부가가치(백만원),총자본투자효율,설비투자효율,기계투자효율,부가가치율,노동소득분배율,자본분배율,이윤분배율,설립일,상장일,상장폐지일,영업활동으로 인한 현금흐름(간접법),투자활동으로 인한 현금흐름,재무활동으로 인한 현금흐름,유동부채,자산,자본,매출액(수익),상장협 산업분류(소분류),상장협 산업분류(중분류),매출액증가율,유형자산증가율,비유동생물자산증가율,투자부동산증가율,유동자산증가율,비유동자산증가율,총자본증가율,자기자본증가율,재고자산증가율,정상영업이익증가율,순이익증가율,총포괄이익증가율,자산.1,부실


In [ ]:
copy.shape

(6507, 130)

In [ ]:
copy['부실'].value_counts()

0.0    6400
1.0     107
Name: 부실, dtype: int64

In [ ]:
# copy - 1,2 전처리 완료된 데이터프레임

제거 완료한 후, (6507, 130) 총 43개 제거됨

##### 비주력산업

In [ ]:
minor2[['회계년도']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4657 entries, 0 to 4859
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   회계년도    4657 non-null   object
dtypes: object(1)
memory usage: 72.8+ KB


In [ ]:
minor2[['상장일']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4657 entries, 0 to 4859
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   상장일     4657 non-null   object
dtypes: object(1)
memory usage: 72.8+ KB


In [ ]:
minor2['회계년도'] = pd.to_datetime(minor2['회계년도'])

In [ ]:
minor2['상장일'] = pd.to_datetime(minor2['상장일'])

In [ ]:
c = minor2[minor2['회계년도'] < minor2['상장일']]

In [ ]:
c.shape

(3, 130)

In [ ]:
minor2.shape

(4657, 130)

In [ ]:
# 회계년도가 상장일 이전에 존재하는 데이터 확인
c[['회사명','거래소코드','회계년도','상장일','부실']]

,회사명,거래소코드,회계년도,상장일,부실
782,빅토리콘텐츠,210120,2017-12-01,2023-08-17,0.0
783,빅토리콘텐츠,210120,2018-12-01,2023-08-17,0.0
784,빅토리콘텐츠,210120,2019-12-01,2023-08-17,0.0


In [ ]:
# 빅토리콘텐츠 코넥스시장에서 2023년에 코스닥시장으로 이전됨 -> 제거

In [ ]:
minor2.reset_index(drop=True, inplace=True)

In [ ]:
# 상장일 이전 회계년도 데이터가 있는 '빅토리콘텐츠' 인덱스 저장
drop_index = minor2[minor2['회사명']== '빅토리콘텐츠'].index
drop_index

Int64Index([782, 783, 784], dtype='int64')

In [ ]:
minor2.drop(drop_index, axis=0, inplace=True) # 빅토리콘텐츠 제거

In [ ]:
minor2[minor2['회사명']== '빅토리콘텐츠']

,회사명,거래소코드,회계년도,매출액총이익률,매출액정상영업이익률,매출액순이익률,총자본사업이익률,총자본정상영업이익률,총자본순이익률,자기자본정상영업이익률,자기자본순이익률,경영자본정상영업이익률,경영자본순이익률,자본금정상영업이익률,자본금순이익률,매출원가 대 매출액비율,영업비용 대 영업수익비율,기타손익비률,금융손익비율,금융비용부담률,외환이익 대 매출액비율,광고선전비 대 매출액비율,세금과공과 대 세금과공과 차감전순이익률,기업순이익률,수지비율(관계기업투자손익 제외),인건비 대 총비용비율,R & D 투자효율,세금과공과 대 총비용비율,금융비용 대 총비용비율,감가상각비 대 총비용비율,감가상각률,누적감가상각률,이자부담률,지급이자율,차입금평균이자율,유보율,적립금비율(재정비율),1주당매출액,1주당순이익,1주당 CASH FLOW,1주당순자산,1주당정상영업이익,유동자산구성비율,재고자산 대 유동자산비율,유동자산 대 비유동자산비율,당좌자산구성비율,비유동자산구성비율,자기자본구성비율,타인자본구성비율,자기자본배율,비유동비율,비유동장기적합률,유동비율,당좌비율,현금비율,매출채권비율,재고자산 대 순운전자본비율,매출채권 대 매입채무비율,매입채무 대 재고자산비율,부채비율,유동부채비율,단기차입금 대 총차입금비율,비유동부채비율,비유동부채 대 순운전자본비율,순운전자본비율,차입금의존도,차입금비율,유보액대비율,유보액 대 납입자본배율,유동자산집중도,비유동자산집중도,투자집중도,CASH FLOW 대 부채비율,CASH FLOW 대 차입금비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,정상영업이익대비이자보상배율,총자본회전률,경영자본회전률,자기자본회전률,자본금회전률,타인자본회전률,매입채무회전률,매입채무회전기간,유동자산회전률,당좌자산회전률,재고자산회전률,재고자산회전기간,재공품회전률,매출채권회전률,매출채권회전기간,비유동자산회전률,유형자산회전율,순운전자본회전률,운전자본회전률,1회전기간,부가가치(백만원),총자본투자효율,설비투자효율,기계투자효율,부가가치율,노동소득분배율,자본분배율,이윤분배율,설립일,상장일,상장폐지일,영업활동으로 인한 현금흐름(간접법),투자활동으로 인한 현금흐름,재무활동으로 인한 현금흐름,유동부채,자산,자본,매출액(수익),상장협 산업분류(소분류),상장협 산업분류(중분류),매출액증가율,유형자산증가율,비유동생물자산증가율,투자부동산증가율,유동자산증가율,비유동자산증가율,자기자본증가율,재고자산증가율,정상영업이익증가율,순이익증가율,총포괄이익증가율,총자본증가율,자산.1,부실


In [ ]:
minor2.shape

(4654, 130)

In [ ]:
minor2['부실'].value_counts()

0.0    4595
1.0      59
Name: 부실, dtype: int64

In [ ]:
# copy - 1,2 전처리 완료된 주력 데이터프레임
# minor2 -1,2 전처리 완료된 비주력 데이터프레임

제거 완료한 후, (4654, 130) 총 3개 제거됨

##### 1,2 전처리 완료한 데이터프레임
  - copy - 1,2 전처리 완료된 주력 데이터프레임
  - minor2 -1,2 전처리 완료된 비주력 데이터프레임

In [ ]:
# copy.to_csv('./drive/MyDrive/공유문서함/상장csv/240315상장_주력전처리중(1,2완료).csv')
# minor2.to_csv('./drive/MyDrive/공유문서함/상장csv/240315상장_비주력전처리중(1,2완료).csv')

#### describe 찍어보기

In [ ]:
new_major = pd.read_csv('./drive/MyDrive/공유문서함/상장csv/240315상장_주력전처리중(1,2완료).csv', index_col=0)
new_minor = pd.read_csv('./drive/MyDrive/공유문서함/상장csv/240315상장_비주력전처리중(1,2완료).csv', index_col=0)

In [ ]:
display(new_major.shape)
display(new_minor.shape)

(6507, 130)

(4654, 130)

In [ ]:
new_major.describe()

,거래소코드,매출액총이익률,매출액정상영업이익률,매출액순이익률,총자본사업이익률,총자본정상영업이익률,총자본순이익률,자기자본정상영업이익률,자기자본순이익률,경영자본정상영업이익률,경영자본순이익률,자본금정상영업이익률,자본금순이익률,매출원가 대 매출액비율,영업비용 대 영업수익비율,기타손익비률,금융손익비율,금융비용부담률,외환이익 대 매출액비율,광고선전비 대 매출액비율,세금과공과 대 세금과공과 차감전순이익률,기업순이익률,수지비율(관계기업투자손익 제외),인건비 대 총비용비율,R & D 투자효율,세금과공과 대 총비용비율,금융비용 대 총비용비율,감가상각비 대 총비용비율,감가상각률,누적감가상각률,이자부담률,지급이자율,차입금평균이자율,유보율,적립금비율(재정비율),1주당매출액,1주당순이익,1주당 CASH FLOW,1주당순자산,1주당정상영업이익,유동자산구성비율,재고자산 대 유동자산비율,유동자산 대 비유동자산비율,당좌자산구성비율,비유동자산구성비율,자기자본구성비율,타인자본구성비율,자기자본배율,비유동비율,비유동장기적합률,유동비율,당좌비율,현금비율,매출채권비율,재고자산 대 순운전자본비율,매출채권 대 매입채무비율,매입채무 대 재고자산비율,부채비율,유동부채비율,단기차입금 대 총차입금비율,비유동부채비율,비유동부채 대 순운전자본비율,순운전자본비율,차입금의존도,차입금비율,유보액대비율,유보액 대 납입자본배율,유동자산집중도,비유동자산집중도,투자집중도,CASH FLOW 대 부채비율,CASH FLOW 대 차입금비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,정상영업이익대비이자보상배율,총자본회전률,경영자본회전률,자기자본회전률,자본금회전률,타인자본회전률,매입채무회전률,매입채무회전기간,유동자산회전률,당좌자산회전률,재고자산회전률,재고자산회전기간,재공품회전률,매출채권회전률,매출채권회전기간,비유동자산회전률,유형자산회전율,순운전자본회전률,운전자본회전률,1회전기간,부가가치(백만원),총자본투자효율,설비투자효율,기계투자효율,부가가치율,노동소득분배율,자본분배율,이윤분배율,영업활동으로 인한 현금흐름(간접법),투자활동으로 인한 현금흐름,재무활동으로 인한 현금흐름,유동부채,자산,자본,매출액(수익),매출액증가율,유형자산증가율,비유동생물자산증가율,투자부동산증가율,유동자산증가율,비유동자산증가율,총자본증가율,자기자본증가율,재고자산증가율,정상영업이익증가율,순이익증가율,총포괄이익증가율,자산.1,부실
count,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6.507000e+03,6507.000000,6507.000000,8.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6.507000e+03,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6.507000e+03,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6.507000e+03,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6.507000e+03,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6.507000e+03,6507.000000,6507.000000,6.507000e+03,6507.000000,6507.000000,6507.000000,6507.000000,6.507000e+03,6.507000e+03,6.507000e+03,6.507000e+03,6.507000e+03,6.507000e+03,6.507000e+03,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6359.000000,6507.000000,6507.000000,6507.000000,6507.000000,6.507000e+03,6507.000000
mean,61802.523436,21.015771,0.676819,-4.253988,3.621497,3.198285,0.511105,4.777143,-3.501411,3.586933,0.268416,146.786501,88.145880,76.648283,99.532925,-3.637501,-1.435585,1.095811,-0.004646,0.805751,14.761821,1.210410,100.742264,15.221634,1.146321,0.119720,0.799119,4.284501,13.607802,82.851030,0.199099,7.295889,5.744140,1517.109588,77.056087,3.973682e+04,1491.508117,3067.781843,2720.657500,2329.433575,51.065912,24.441830,143.880996,39.142305,48.690092,55.602313,44.490148,1562.868419,127.808606,82.280272,248.884836,200.401252,53.842004,1.435475e+03,291.024640,412.871924,727.490602,148.386113,105.335002,51.302170,40.793275,5.359418e+02,17.381405,23.555863,85.171903,48.178139,16.294872,76.200314,78.610438,129.290269,16.948463,1.235629e+03,4.201777,3.328019,19.956495,0.914689,1.057470,2.016003,25.970899,2.625216,24.332110,35.970589,1.879596,2.657191,84.069333,55.863015,1.488754e+03,7.457495,73.225105,2.287993,6.794846,1.095494,1.517872,0.024289,3.414235e+05,17.331654,144.800092,2.006104e+05,19.614621,93.598156,-0.836515,-38.914627,1.519755e+08,-1.448830e+08,1.330935e+06,5.076727e+08,1.876604e+09,9.413415e+08,1.537311e+09,0.090295,0.658372,0.000429,0.337514,0.091702,0.144585,0.109406,inf,0.446908,-0.329679,-5.497993,-0.469576,1.876604e+09,0.016444
std,54716.997323,18.282104,59.988057,69.913204,8.620701,8.598316,13.232408,22.422076,

#### 3. 자기자본증가율, 총자본증가율 수정

자기자본증가율은 자본항목으로 계산하면 되고,
총자본증가율은 자본항목 + 부채항목으로 계산하면 됨

##### 주력산업

In [ ]:
df_1 = pd.read_csv("./drive/MyDrive/공유문서함/상장csv/상장주력_연결(참고용).csv",index_col=0)
df_2 = pd.read_csv("./drive/MyDrive/공유문서함/상장csv/상장주력_개별(참고용).csv",index_col=0)

In [ ]:
df_1.isnull().sum()

회사명                                 0
거래소코드                               0
회계년도                                0
상장협 산업분류(소분류)                       0
상장협 산업분류(중분류)                       0
자본(*)(IFRS연결)(천원)_y              2756
유형자산(*)(IFRS연결)(천원)              2756
비유동생물자산(*)(IFRS연결)(천원)           2756
투자부동산(*)(IFRS연결)(천원)             2756
비유동자산(*)(IFRS연결)(천원)             2756
유동자산(*)(IFRS연결)(천원)              2756
재고자산(*)(IFRS연결)(천원)              2756
* (정상)영업손익(계산수치)(IFRS연결)(천원)     2756
* (정상)영업손익(보고서기재)(IFRS연결)(천원)    2756
당기순이익(손실)(IFRS연결)(천원)            2756
총포괄손익(IFRS연결)(천원)                2756
매출액(수익)(*)(IFRS연결)(천원)_y         2756
부채(*)(IFRS연결)(천원)                2756
dtype: int64

In [ ]:
df_2.drop(['자본(*)(IFRS연결)(천원)','매출액(수익)(*)(IFRS연결)(천원)'], axis=1, inplace=True)

In [ ]:
df_1.columns = df_1.columns.str.replace('y','')
df_1.columns = df_1.columns.str.replace('_','')
df_2.columns= df_2.columns.str.replace('y','')
df_2.columns=df_2.columns.str.replace('_','')

In [ ]:
df_1.isnull().sum()

회사명                                 0
거래소코드                               0
회계년도                                0
상장협 산업분류(소분류)                       0
상장협 산업분류(중분류)                       0
자본(*)(IFRS연결)(천원)                2756
유형자산(*)(IFRS연결)(천원)              2756
비유동생물자산(*)(IFRS연결)(천원)           2756
투자부동산(*)(IFRS연결)(천원)             2756
비유동자산(*)(IFRS연결)(천원)             2756
유동자산(*)(IFRS연결)(천원)              2756
재고자산(*)(IFRS연결)(천원)              2756
* (정상)영업손익(계산수치)(IFRS연결)(천원)     2756
* (정상)영업손익(보고서기재)(IFRS연결)(천원)    2756
당기순이익(손실)(IFRS연결)(천원)            2756
총포괄손익(IFRS연결)(천원)                2756
매출액(수익)(*)(IFRS연결)(천원)           2756
부채(*)(IFRS연결)(천원)                2756
dtype: int64

In [ ]:
#컬럼명 일치를 위한 컬럼명 변경
df_1.columns = df_1.columns.str.replace('연결','')
df_1.columns

Index(['회사명', '거래소코드', '회계년도', '상장협 산업분류(소분류)', '상장협 산업분류(중분류)',
       '자본(*)(IFRS)(천원)', '유형자산(*)(IFRS)(천원)', '비유동생물자산(*)(IFRS)(천원)',
       '투자부동산(*)(IFRS)(천원)', '비유동자산(*)(IFRS)(천원)', '유동자산(*)(IFRS)(천원)',
       '재고자산(*)(IFRS)(천원)', '* (정상)영업손익(계산수치)(IFRS)(천원)',
       '* (정상)영업손익(보고서기재)(IFRS)(천원)', '당기순이익(손실)(IFRS)(천원)', '총포괄손익(IFRS)(천원)',
       '매출액(수익)(*)(IFRS)(천원)', '부채(*)(IFRS)(천원)'],
      dtype='object')

In [ ]:
list = ['자본(*)(IFRS)(천원)', '유형자산(*)(IFRS)(천원)', '비유동생물자산(*)(IFRS)(천원)',
       '투자부동산(*)(IFRS)(천원)', '비유동자산(*)(IFRS)(천원)', '유동자산(*)(IFRS)(천원)',
       '재고자산(*)(IFRS)(천원)', '* (정상)영업손익(계산수치)(IFRS)(천원)',
       '* (정상)영업손익(보고서기재)(IFRS)(천원)', '당기순이익(손실)(IFRS)(천원)', '총포괄손익(IFRS)(천원)',
       '매출액(수익)(*)(IFRS)(천원)', '부채(*)(IFRS)(천원)']

In [ ]:
# 빈공간을 개별로 넣기
for i in list:
    df_1[i] = df_1[i].fillna(df_2[i])

In [ ]:
df_1.dropna(inplace=True)

In [ ]:
df_1.reset_index(drop=True,inplace=True)

In [ ]:
new_major.drop(['자기자본증가율', '총자본증가율','자산.1'], axis=1, inplace=True)

총자본증가율

In [ ]:
df_1["자산"] = df_1["자본(*)(IFRS)(천원)"] +  df_1["부채(*)(IFRS)(천원)"]
df_1

,회사명,거래소코드,회계년도,상장협 산업분류(소분류),상장협 산업분류(중분류),자본(*)(IFRS)(천원),유형자산(*)(IFRS)(천원),비유동생물자산(*)(IFRS)(천원),투자부동산(*)(IFRS)(천원),비유동자산(*)(IFRS)(천원),유동자산(*)(IFRS)(천원),재고자산(*)(IFRS)(천원),* (정상)영업손익(계산수치)(IFRS)(천원),* (정상)영업손익(보고서기재)(IFRS)(천원),당기순이익(손실)(IFRS)(천원),총포괄손익(IFRS)(천원),매출액(수익)(*)(IFRS)(천원),부채(*)(IFRS)(천원),자산
0,(주)CMG제약,58820,2010/12,의약품 제조업,의료용 물질 및 의약품 제조업,18405116.0,10316860.0,0.0,0.0,13734412.0,13041215.0,2782759.0,-6749246.0,-7537071.0,-6945220.0,-6979964.0,11350419.0,8370511.0,26775627.0
1,(주)CMG제약,58820,2011/12,의약품 제조업,의료용 물질 및 의약품 제조업,18034249.0,9707759.0,0.0,0.0,11964031.0,15341167.0,2743103.0,623816.0,640283.0,-1508282.0,-1432267.0,17751704.0,9270949.0,27305198.0
2,(주)CMG제약,58820,2012/12,의약품 제조업,의료용 물질 및 의약품 제조업,31590685.0,9307482.0,0.0,0.0,9998822.0,47109213.0,2955172.0,-4205503.0,-4205503.0,-6660149.0,-7206864.0,16255389.0,25517350.0,57108035.0
3,(주)CMG제약,58820,2013/12,의약품 제조업,의료용 물질 및 의약품 제조업,48997281.0,10805092.0,0.0,0.0,17702864.0,38209699.0,4618718.0,791250.0,791250.0,296181.0,178271.0,20402140.0,6915281.0,55912562.0
4,(주)CMG제약,58820,2014/12,의약품 제조업,의료용 물질 및 의약품 제조업,49002644.0,18833775.0,0.0,0.0,29767385.0,28300941.0,4913103.0,442322.0,442322.0,158039.0,-192199.0,22752584.0,9065682.0,58068326.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10308,휴림로봇(주),90710,2015/12,특수 목적용 기계 제조업,기타 기계 및 장비 제조업,10283583.0,7609902.0,0.0,0.0,10229802.0,15940616.0,3673337.0,1445320.0,1445320.0,1278796.0,558572.0,27151771.0,15886836.0,26170419.0
10309,휴림로봇(주),90710,2016/12,특수 목적용 기계 제조업,기타 기계 및 장비 제조업,13455393.0,7286127.0,0.0,0.0,9625647.0,32990505.0,4339597.0,1625226.0,1625226.0,1091841.0,1159385.0,45094012.0,29160759.0,42616152.0
10310,휴림로봇(주),90710,2017/12,특수 목적용 기계 제조업,기타 기계 및 장비 제조업,24606091.0,7303959.0,0.0,0.0,42760901.0,31635683.0,3793192.0,4369941.0,4369941.0,1742450.0,100887.0,71499983.0,49790492.0,74396583.0
10311,휴림로봇(주),90710,2018/12,특수 목적용 기계 제조업,기타 기계 및 장비 제조업,25445610.0,8446292.0,0.0,0.0,17443238.0,28137457.0,2986180.0,-8062766.0,-8062766.0,-19026771.0,-18544589.0,30729831.0,20135085.0,45580695.0


In [ ]:
# 데이터에 0 이 있기 때문에 계산데이터에 0이 들어있으면 증가율 데이터는 무조건 0처리
for i in df_1.index:
    if i > 0:
        if df_1.loc[i,"회사명"] == df_1.loc[i-1,"회사명"]:
            if df_1.loc[i-1,"자산"] <= 0:
                df_1.loc[i,"총자본증가율"] = 0
            else:
                df_1.loc[i,"총자본증가율"] = (df_1.loc[i,"자산"]-df_1.loc[i-1,"자산"])/df_1.loc[i-1,"자산"]
        elif df_1.loc[i,"회사명"] != df_1.loc[i-1,"회사명"]:
            df_1.loc[i,"총자본증가율"] = np.NAN

자기자본증가율

In [ ]:
for i in df_1.index:
    if i > 0:
        if df_1.loc[i,"회사명"] == df_1.loc[i-1,"회사명"]:
            if df_1.loc[i-1,"자본(*)(IFRS)(천원)"] <= 0:
                df_1.loc[i,"자기자본증가율"] = 0
            else:
                df_1.loc[i,"자기자본증가율"] = (df_1.loc[i,"자본(*)(IFRS)(천원)"]-df_1.loc[i-1,"자본(*)(IFRS)(천원)"])/df_1.loc[i-1,"자본(*)(IFRS)(천원)"]
        elif df_1.loc[i,"회사명"] != df_1.loc[i-1,"회사명"]:
            df_1.loc[i,"자기자본증가율"] = np.NAN

합치기

In [ ]:
# (주)회사명에서 (주)를 제거_주력산업
df_1.loc[df_1['회사명'].str.contains('\(주\)'), '회사명'] = df_1.loc[df_1['회사명'].str.contains('\(주\)'), '회사명'].str.replace('\(주\)', '')

df_1.head()

,회사명,거래소코드,회계년도,상장협 산업분류(소분류),상장협 산업분류(중분류),자본(*)(IFRS)(천원),유형자산(*)(IFRS)(천원),비유동생물자산(*)(IFRS)(천원),투자부동산(*)(IFRS)(천원),비유동자산(*)(IFRS)(천원),유동자산(*)(IFRS)(천원),재고자산(*)(IFRS)(천원),* (정상)영업손익(계산수치)(IFRS)(천원),* (정상)영업손익(보고서기재)(IFRS)(천원),당기순이익(손실)(IFRS)(천원),총포괄손익(IFRS)(천원),매출액(수익)(*)(IFRS)(천원),부채(*)(IFRS)(천원),자산,총자본증가율,자기자본증가율
0,CMG제약,58820,2010/12,의약품 제조업,의료용 물질 및 의약품 제조업,18405116.0,10316860.0,0.0,0.0,13734412.0,13041215.0,2782759.0,-6749246.0,-7537071.0,-6945220.0,-6979964.0,11350419.0,8370511.0,26775627.0,NaN,NaN
1,CMG제약,58820,2011/12,의약품 제조업,의료용 물질 및 의약품 제조업,18034249.0,9707759.0,0.0,0.0,11964031.0,15341167.0,2743103.0,623816.0,640283.0,-1508282.0,-1432267.0,17751704.0,9270949.0,27305198.0,0.019778,-0.020150
2,CMG제약,58820,2012/12,의약품 제조업,의료용 물질 및 의약품 제조업,31590685.0,9307482.0,0.0,0.0,9998822.0,47109213.0,2955172.0,-4205503.0,-4205503.0,-6660149.0,-7206864.0,16255389.0,25517350.0,57108035.0,1.091471,0.751705
3,CMG제약,58820,2013/12,의약품 제조업,의료용 물질 및 의약품 제조업,48997281.0,10805092.0,0.0,0.0,17702864.0,38209699.0,4618718.0,791250.0,791250.0,296181.0,178271.0,20402140.0,6915281.0,55912562.0,-0.020934,0.551004
4,CMG제약,58820,2014/12,의약품 제조업,의료용 물질 및 의약품 제조업,49002644.0,18833775.0,0.0,0.0,29767385.0,28300941.0,4913103.0,442322.0,442322.0,158039.0,-192199.0,22752584.0,9065682.0,58068326.0,0.038556,0.000109


In [ ]:
df_1.head(1)

,회사명,거래소코드,회계년도,상장협 산업분류(소분류),상장협 산업분류(중분류),자본(*)(IFRS)(천원),유형자산(*)(IFRS)(천원),비유동생물자산(*)(IFRS)(천원),투자부동산(*)(IFRS)(천원),비유동자산(*)(IFRS)(천원),유동자산(*)(IFRS)(천원),재고자산(*)(IFRS)(천원),* (정상)영업손익(계산수치)(IFRS)(천원),* (정상)영업손익(보고서기재)(IFRS)(천원),당기순이익(손실)(IFRS)(천원),총포괄손익(IFRS)(천원),매출액(수익)(*)(IFRS)(천원),부채(*)(IFRS)(천원),자산,총자본증가율,자기자본증가율
0,CMG제약,58820,2010/12,의약품 제조업,의료용 물질 및 의약품 제조업,18405116.0,10316860.0,0.0,0.0,13734412.0,13041215.0,2782759.0,-6749246.0,-7537071.0,-6945220.0,-6979964.0,11350419.0,8370511.0,26775627.0,NaN,NaN


In [ ]:
# 날짜데이터 datetime화
new_major["회계년도"] = pd.to_datetime(new_major["회계년도"])
df_1["회계년도"] = pd.to_datetime(df_1["회계년도"])

In [ ]:
new_major.shape

(6507, 128)

In [ ]:
df_all = pd.merge(new_major, df_1, how='left', on = ['회사명','거래소코드','회계년도'])

In [ ]:
df_all.shape

(6507, 146)

In [ ]:
df_all.columns.tolist()

['회사명',
 '거래소코드',
 '회계년도',
 '매출액총이익률',
 '매출액정상영업이익률',
 '매출액순이익률',
 '총자본사업이익률',
 '총자본정상영업이익률',
 '총자본순이익률',
 '자기자본정상영업이익률',
 '자기자본순이익률',
 '경영자본정상영업이익률',
 '경영자본순이익률',
 '자본금정상영업이익률',
 '자본금순이익률',
 '매출원가 대 매출액비율',
 '영업비용 대 영업수익비율',
 '기타손익비률',
 '금융손익비율',
 '금융비용부담률',
 '외환이익 대 매출액비율',
 '광고선전비 대 매출액비율',
 '세금과공과 대 세금과공과 차감전순이익률',
 '기업순이익률',
 '수지비율(관계기업투자손익 제외)',
 '인건비 대 총비용비율',
 'R & D 투자효율',
 '세금과공과 대 총비용비율',
 '금융비용 대 총비용비율',
 '감가상각비 대 총비용비율',
 '감가상각률',
 '누적감가상각률',
 '이자부담률',
 '지급이자율',
 '차입금평균이자율',
 '유보율',
 '적립금비율(재정비율)',
 '1주당매출액',
 '1주당순이익',
 '1주당 CASH FLOW',
 '1주당순자산',
 '1주당정상영업이익',
 '유동자산구성비율',
 '재고자산 대 유동자산비율',
 '유동자산 대 비유동자산비율',
 '당좌자산구성비율',
 '비유동자산구성비율',
 '자기자본구성비율',
 '타인자본구성비율',
 '자기자본배율',
 '비유동비율',
 '비유동장기적합률',
 '유동비율',
 '당좌비율',
 '현금비율',
 '매출채권비율',
 '재고자산 대 순운전자본비율',
 '매출채권 대 매입채무비율',
 '매입채무 대 재고자산비율',
 '부채비율',
 '유동부채비율',
 '단기차입금 대 총차입금비율',
 '비유동부채비율',
 '비유동부채 대 순운전자본비율',
 '순운전자본비율',
 '차입금의존도',
 '차입금비율',
 '유보액대비율',
 '유보액 대 납입자본배율',
 '유동자산집중도',
 '비유동자산집중도',
 '투자집중도',
 'CASH FLOW 대 부채비율',


In [ ]:
a = df_all.copy()

In [ ]:
a.drop(['총자본증가율_x',
 '상장협 산업분류(소분류)_y',
 '상장협 산업분류(중분류)_y',
 '자본(*)(IFRS)(천원)',
 '유형자산(*)(IFRS)(천원)',
 '비유동생물자산(*)(IFRS)(천원)',
 '투자부동산(*)(IFRS)(천원)',
 '비유동자산(*)(IFRS)(천원)',
 '유동자산(*)(IFRS)(천원)',
 '재고자산(*)(IFRS)(천원)',
 '* (정상)영업손익(계산수치)(IFRS)(천원)',
 '* (정상)영업손익(보고서기재)(IFRS)(천원)',
 '당기순이익(손실)(IFRS)(천원)',
 '총포괄손익(IFRS)(천원)',
 '매출액(수익)(*)(IFRS)(천원)',
 '부채(*)(IFRS)(천원)',
 '자산_y'],axis=1,inplace=True)

In [ ]:
a.shape

(6507, 129)

In [ ]:
a.describe()

,거래소코드,매출액총이익률,매출액정상영업이익률,매출액순이익률,총자본사업이익률,총자본정상영업이익률,총자본순이익률,자기자본정상영업이익률,자기자본순이익률,경영자본정상영업이익률,경영자본순이익률,자본금정상영업이익률,자본금순이익률,매출원가 대 매출액비율,영업비용 대 영업수익비율,기타손익비률,금융손익비율,금융비용부담률,외환이익 대 매출액비율,광고선전비 대 매출액비율,세금과공과 대 세금과공과 차감전순이익률,기업순이익률,수지비율(관계기업투자손익 제외),인건비 대 총비용비율,R & D 투자효율,세금과공과 대 총비용비율,금융비용 대 총비용비율,감가상각비 대 총비용비율,감가상각률,누적감가상각률,이자부담률,지급이자율,차입금평균이자율,유보율,적립금비율(재정비율),1주당매출액,1주당순이익,1주당 CASH FLOW,1주당순자산,1주당정상영업이익,유동자산구성비율,재고자산 대 유동자산비율,유동자산 대 비유동자산비율,당좌자산구성비율,비유동자산구성비율,자기자본구성비율,타인자본구성비율,자기자본배율,비유동비율,비유동장기적합률,유동비율,당좌비율,현금비율,매출채권비율,재고자산 대 순운전자본비율,매출채권 대 매입채무비율,매입채무 대 재고자산비율,부채비율,유동부채비율,단기차입금 대 총차입금비율,비유동부채비율,비유동부채 대 순운전자본비율,순운전자본비율,차입금의존도,차입금비율,유보액대비율,유보액 대 납입자본배율,유동자산집중도,비유동자산집중도,투자집중도,CASH FLOW 대 부채비율,CASH FLOW 대 차입금비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,정상영업이익대비이자보상배율,총자본회전률,경영자본회전률,자기자본회전률,자본금회전률,타인자본회전률,매입채무회전률,매입채무회전기간,유동자산회전률,당좌자산회전률,재고자산회전률,재고자산회전기간,재공품회전률,매출채권회전률,매출채권회전기간,비유동자산회전률,유형자산회전율,순운전자본회전률,운전자본회전률,1회전기간,부가가치(백만원),총자본투자효율,설비투자효율,기계투자효율,부가가치율,노동소득분배율,자본분배율,이윤분배율,영업활동으로 인한 현금흐름(간접법),투자활동으로 인한 현금흐름,재무활동으로 인한 현금흐름,유동부채,자산_x,자본,매출액(수익),매출액증가율,유형자산증가율,비유동생물자산증가율,투자부동산증가율,유동자산증가율,비유동자산증가율,재고자산증가율,정상영업이익증가율,순이익증가율,총포괄이익증가율,부실,총자본증가율,자기자본증가율
count,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6.507000e+03,6507.000000,6507.000000,8.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6.507000e+03,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6.507000e+03,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6.507000e+03,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6.507000e+03,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6.507000e+03,6507.000000,6507.000000,6.507000e+03,6507.000000,6507.000000,6507.000000,6507.000000,6.507000e+03,6.507000e+03,6.507000e+03,6.507000e+03,6.507000e+03,6.507000e+03,6.507000e+03,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000
mean,61802.523436,21.015771,0.676819,-4.253988,3.621497,3.198285,0.511105,4.777143,-3.501411,3.586933,0.268416,146.786501,88.145880,76.648283,99.532925,-3.637501,-1.435585,1.095811,-0.004646,0.805751,14.761821,1.210410,100.742264,15.221634,1.146321,0.119720,0.799119,4.284501,13.607802,82.851030,0.199099,7.295889,5.744140,1517.109588,77.056087,3.973682e+04,1491.508117,3067.781843,2720.657500,2329.433575,51.065912,24.441830,143.880996,39.142305,48.690092,55.602313,44.490148,1562.868419,127.808606,82.280272,248.884836,200.401252,53.842004,1.435475e+03,291.024640,412.871924,727.490602,148.386113,105.335002,51.302170,40.793275,5.359418e+02,17.381405,23.555863,85.171903,48.178139,16.294872,76.200314,78.610438,129.290269,16.948463,1.235629e+03,4.201777,3.328019,19.956495,0.914689,1.057470,2.016003,25.970899,2.625216,24.332110,35.970589,1.879596,2.657191,84.069333,55.863015,1.488754e+03,7.457495,73.225105,2.287993,6.794846,1.095494,1.517872,0.024289,3.414235e+05,17.331654,144.800092,2.006104e+05,19.614621,93.598156,-0.836515,-38.914627,1.519755e+08,-1.448830e+08,1.330935e+06,5.076727e+08,1.876604e+09,9.413415e+08,1.537311e+09,0.090295,0.658372,0.000429,0.337514,0.091702,0.144585,0.446908,-0.329679,-5.497993,-0.469576,0.016444,0.083767,0.109406
std,54716.997323,18.282104,59.988057,69.913204,8.620701,8.598316,13.232408,22.422076,65.486517,11.436553,19.2

In [ ]:
a.rename(columns={'총자본증가율_y':'총자본증가율'}, inplace=True)

In [ ]:
# a: 총자본증가율과 자기자본증가율 수정한 데이터프레임
# a.to_csv('./drive/MyDrive/공유문서함/상장csv/240316상장_주력전처리중(1,2,3완료).csv')

##### 비주력산업

In [ ]:
df_1 = pd.read_csv("./drive/MyDrive/공유문서함/상장csv/상장비주력_연결(참고용).csv",index_col=0)
df_2 = pd.read_csv("./drive/MyDrive/공유문서함/상장csv/상장비주력_개별(참고용).csv",index_col=0)

In [ ]:
df_1.isnull().sum()

회사명                                 0
거래소코드                               0
회계년도                                0
상장협 산업분류(소분류)                       0
상장협 산업분류(중분류)                       0
자본(*)(IFRS연결)(천원)_y              1608
유형자산(*)(IFRS연결)(천원)              1608
비유동생물자산(*)(IFRS연결)(천원)           1608
투자부동산(*)(IFRS연결)(천원)             1608
비유동자산(*)(IFRS연결)(천원)             1608
유동자산(*)(IFRS연결)(천원)              1608
재고자산(*)(IFRS연결)(천원)              1608
* (정상)영업손익(계산수치)(IFRS연결)(천원)     1608
* (정상)영업손익(보고서기재)(IFRS연결)(천원)    1608
당기순이익(손실)(IFRS연결)(천원)            1608
총포괄손익(IFRS연결)(천원)                1608
매출액(수익)(*)(IFRS연결)(천원)_y         1608
부채(*)(IFRS연결)(천원)                1608
dtype: int64

In [ ]:
df_1.columns = df_1.columns.str.replace('y','')
df_1.columns = df_1.columns.str.replace('_','')
df_2.columns= df_2.columns.str.replace('y','')
df_2.columns=df_2.columns.str.replace('_','')

In [ ]:
df_2.isnull().sum()

회사명                              0
거래소코드                            0
회계년도                             0
자본(*)(IFRS)(천원)                250
유형자산(*)(IFRS)(천원)              250
투자부동산(*)(IFRS)(천원)             250
비유동생물자산(*)(IFRS)(천원)           250
비유동자산(*)(IFRS)(천원)             250
유동자산(*)(IFRS)(천원)              250
재고자산(*)(IFRS)(천원)              250
부채(*)(IFRS)(천원)                250
매출액(수익)(*)(IFRS)(천원)           250
* (정상)영업손익(계산수치)(IFRS)(천원)     250
* (정상)영업손익(보고서기재)(IFRS)(천원)    250
당기순이익(손실)(IFRS)(천원)            250
총포괄손익(IFRS)(천원)                250
dtype: int64

In [ ]:
#컬럼명 일치를 위한 컬럼명 변경
df_1.columns = df_1.columns.str.replace('연결','')
df_1.columns

Index(['회사명', '거래소코드', '회계년도', '상장협 산업분류(소분류)', '상장협 산업분류(중분류)',
       '자본(*)(IFRS)(천원)', '유형자산(*)(IFRS)(천원)', '비유동생물자산(*)(IFRS)(천원)',
       '투자부동산(*)(IFRS)(천원)', '비유동자산(*)(IFRS)(천원)', '유동자산(*)(IFRS)(천원)',
       '재고자산(*)(IFRS)(천원)', '* (정상)영업손익(계산수치)(IFRS)(천원)',
       '* (정상)영업손익(보고서기재)(IFRS)(천원)', '당기순이익(손실)(IFRS)(천원)', '총포괄손익(IFRS)(천원)',
       '매출액(수익)(*)(IFRS)(천원)', '부채(*)(IFRS)(천원)'],
      dtype='object')

In [ ]:
list = ['자본(*)(IFRS)(천원)', '유형자산(*)(IFRS)(천원)', '비유동생물자산(*)(IFRS)(천원)',
       '투자부동산(*)(IFRS)(천원)', '비유동자산(*)(IFRS)(천원)', '유동자산(*)(IFRS)(천원)',
       '재고자산(*)(IFRS)(천원)', '* (정상)영업손익(계산수치)(IFRS)(천원)',
       '* (정상)영업손익(보고서기재)(IFRS)(천원)', '당기순이익(손실)(IFRS)(천원)', '총포괄손익(IFRS)(천원)',
       '매출액(수익)(*)(IFRS)(천원)', '부채(*)(IFRS)(천원)']

In [ ]:
# 빈공간을 개별로 넣기
for i in list:
    df_1[i] = df_1[i].fillna(df_2[i])

In [ ]:
df_1.dropna(inplace=True)

In [ ]:
df_1.reset_index(drop=True,inplace=True)

In [ ]:
new_minor.drop(['자기자본증가율', '총자본증가율','자산.1'], axis=1, inplace=True)

총자본증가율

In [ ]:
df_1["자산"] = df_1["자본(*)(IFRS)(천원)"] +  df_1["부채(*)(IFRS)(천원)"]
df_1

,회사명,거래소코드,회계년도,상장협 산업분류(소분류),상장협 산업분류(중분류),자본(*)(IFRS)(천원),유형자산(*)(IFRS)(천원),비유동생물자산(*)(IFRS)(천원),투자부동산(*)(IFRS)(천원),비유동자산(*)(IFRS)(천원),유동자산(*)(IFRS)(천원),재고자산(*)(IFRS)(천원),* (정상)영업손익(계산수치)(IFRS)(천원),* (정상)영업손익(보고서기재)(IFRS)(천원),당기순이익(손실)(IFRS)(천원),총포괄손익(IFRS)(천원),매출액(수익)(*)(IFRS)(천원),부채(*)(IFRS)(천원),자산
0,(주)ES큐브,50120,2010/12,생활용품 도매업,도매 및 상품 중개업,42621100.0,3374904.0,0.0,0.0,16368111.0,32820950.0,4166815.0,-533613.0,-533613.0,-1632071.0,-1517812.0,43887335.0,6567960.0,49189060.0
1,(주)ES큐브,50120,2011/12,생활용품 도매업,도매 및 상품 중개업,39245830.0,4313834.0,0.0,0.0,21398837.0,34078195.0,5312339.0,-3650245.0,-3650245.0,-4796438.0,-4521087.0,53379952.0,16231202.0,55477032.0
2,(주)ES큐브,50120,2012/12,생활용품 도매업,도매 및 상품 중개업,39859681.0,3181598.0,0.0,0.0,23574654.0,22707006.0,4844206.0,89337.0,89337.0,-359216.0,-750162.0,55971147.0,6421979.0,46281660.0
3,(주)ES큐브,50120,2013/12,생활용품 도매업,도매 및 상품 중개업,42372440.0,2177498.0,0.0,0.0,8780362.0,38357211.0,6264936.0,-1164611.0,-1164611.0,-16455910.0,-15845515.0,58597361.0,4765132.0,47137572.0
4,(주)ES큐브,50120,2014/12,생활용품 도매업,도매 및 상품 중개업,49628091.0,2299195.0,0.0,0.0,10397078.0,46214758.0,6084161.0,171090.0,171090.0,-615409.0,-729125.0,44459581.0,6983745.0,56611836.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7158,흥아해운(주),3280,2015/12,해상 운송업,수상 운송업,173076634.0,483319774.0,0.0,45367207.0,655360545.0,136280257.0,11488626.0,21244278.0,21244278.0,10517216.0,6425448.0,845115359.0,618564168.0,791640802.0
7159,흥아해운(주),3280,2016/12,해상 운송업,수상 운송업,192487245.0,671042170.0,0.0,46673216.0,856428431.0,102103828.0,17967554.0,5895873.0,5895873.0,-17173253.0,-24165133.0,831746081.0,766045015.0,958532260.0
7160,흥아해운(주),3280,2017/12,해상 운송업,수상 운송업,116884496.0,560312598.0,0.0,41247303.0,733283507.0,115675013.0,19976759.0,-13098624.0,-13098624.0,-73500411.0,-74970182.0,836427496.0,732074025.0,848958521.0
7161,흥아해운(주),3280,2018/12,해상 운송업,수상 운송업,73148175.0,492709259.0,0.0,70352339.0,695343474.0,108354984.0,18807927.0,-37595967.0,-37595967.0,-86540156.0,-88306302.0,753865569.0,730550282.0,803698457.0


In [ ]:
# 데이터에 0 이 있기 때문에 계산데이터에 0이 들어있으면 증가율 데이터는 무조건 0처리
for i in df_1.index:
    if i > 0:
        if df_1.loc[i,"회사명"] == df_1.loc[i-1,"회사명"]:
            if df_1.loc[i-1,"자산"] <= 0:
                df_1.loc[i,"총자본증가율"] = 0
            else:
                df_1.loc[i,"총자본증가율"] = (df_1.loc[i,"자산"]-df_1.loc[i-1,"자산"])/df_1.loc[i-1,"자산"]
        elif df_1.loc[i,"회사명"] != df_1.loc[i-1,"회사명"]:
            df_1.loc[i,"총자본증가율"] = np.NAN

자기자본증가율

In [ ]:
for i in df_1.index:
    if i > 0:
        if df_1.loc[i,"회사명"] == df_1.loc[i-1,"회사명"]:
            if df_1.loc[i-1,"자본(*)(IFRS)(천원)"] <= 0:
                df_1.loc[i,"자기자본증가율"] = 0
            else:
                df_1.loc[i,"자기자본증가율"] = (df_1.loc[i,"자본(*)(IFRS)(천원)"]-df_1.loc[i-1,"자본(*)(IFRS)(천원)"])/df_1.loc[i-1,"자본(*)(IFRS)(천원)"]
        elif df_1.loc[i,"회사명"] != df_1.loc[i-1,"회사명"]:
            df_1.loc[i,"자기자본증가율"] = np.NAN

합치기

In [ ]:
# (주)회사명에서 (주)를 제거_주력산업
df_1.loc[df_1['회사명'].str.contains('\(주\)'), '회사명'] = df_1.loc[df_1['회사명'].str.contains('\(주\)'), '회사명'].str.replace('\(주\)', '')

df_1.head()

,회사명,거래소코드,회계년도,상장협 산업분류(소분류),상장협 산업분류(중분류),자본(*)(IFRS)(천원),유형자산(*)(IFRS)(천원),비유동생물자산(*)(IFRS)(천원),투자부동산(*)(IFRS)(천원),비유동자산(*)(IFRS)(천원),유동자산(*)(IFRS)(천원),재고자산(*)(IFRS)(천원),* (정상)영업손익(계산수치)(IFRS)(천원),* (정상)영업손익(보고서기재)(IFRS)(천원),당기순이익(손실)(IFRS)(천원),총포괄손익(IFRS)(천원),매출액(수익)(*)(IFRS)(천원),부채(*)(IFRS)(천원),자산,총자본증가율,자기자본증가율
0,ES큐브,50120,2010/12,생활용품 도매업,도매 및 상품 중개업,42621100.0,3374904.0,0.0,0.0,16368111.0,32820950.0,4166815.0,-533613.0,-533613.0,-1632071.0,-1517812.0,43887335.0,6567960.0,49189060.0,NaN,NaN
1,ES큐브,50120,2011/12,생활용품 도매업,도매 및 상품 중개업,39245830.0,4313834.0,0.0,0.0,21398837.0,34078195.0,5312339.0,-3650245.0,-3650245.0,-4796438.0,-4521087.0,53379952.0,16231202.0,55477032.0,0.127833,-0.079192
2,ES큐브,50120,2012/12,생활용품 도매업,도매 및 상품 중개업,39859681.0,3181598.0,0.0,0.0,23574654.0,22707006.0,4844206.0,89337.0,89337.0,-359216.0,-750162.0,55971147.0,6421979.0,46281660.0,-0.165751,0.015641
3,ES큐브,50120,2013/12,생활용품 도매업,도매 및 상품 중개업,42372440.0,2177498.0,0.0,0.0,8780362.0,38357211.0,6264936.0,-1164611.0,-1164611.0,-16455910.0,-15845515.0,58597361.0,4765132.0,47137572.0,0.018494,0.063040
4,ES큐브,50120,2014/12,생활용품 도매업,도매 및 상품 중개업,49628091.0,2299195.0,0.0,0.0,10397078.0,46214758.0,6084161.0,171090.0,171090.0,-615409.0,-729125.0,44459581.0,6983745.0,56611836.0,0.200992,0.171235


In [ ]:
df_1.head(1)

,회사명,거래소코드,회계년도,상장협 산업분류(소분류),상장협 산업분류(중분류),자본(*)(IFRS)(천원),유형자산(*)(IFRS)(천원),비유동생물자산(*)(IFRS)(천원),투자부동산(*)(IFRS)(천원),비유동자산(*)(IFRS)(천원),유동자산(*)(IFRS)(천원),재고자산(*)(IFRS)(천원),* (정상)영업손익(계산수치)(IFRS)(천원),* (정상)영업손익(보고서기재)(IFRS)(천원),당기순이익(손실)(IFRS)(천원),총포괄손익(IFRS)(천원),매출액(수익)(*)(IFRS)(천원),부채(*)(IFRS)(천원),자산,총자본증가율,자기자본증가율
0,ES큐브,50120,2010/12,생활용품 도매업,도매 및 상품 중개업,42621100.0,3374904.0,0.0,0.0,16368111.0,32820950.0,4166815.0,-533613.0,-533613.0,-1632071.0,-1517812.0,43887335.0,6567960.0,49189060.0,NaN,NaN


In [ ]:
# 날짜데이터 datetime화
new_minor["회계년도"] = pd.to_datetime(new_minor["회계년도"])
df_1["회계년도"] = pd.to_datetime(df_1["회계년도"])

In [ ]:
new_minor.shape

(4654, 127)

In [ ]:
df_all = pd.merge(new_minor, df_1, how='left', on = ['회사명','거래소코드','회계년도'])

In [ ]:
df_all.shape

(4654, 145)

In [ ]:
df_all.columns.tolist()

['회사명',
 '거래소코드',
 '회계년도',
 '매출액총이익률',
 '매출액정상영업이익률',
 '매출액순이익률',
 '총자본사업이익률',
 '총자본정상영업이익률',
 '총자본순이익률',
 '자기자본정상영업이익률',
 '자기자본순이익률',
 '경영자본정상영업이익률',
 '경영자본순이익률',
 '자본금정상영업이익률',
 '자본금순이익률',
 '매출원가 대 매출액비율',
 '영업비용 대 영업수익비율',
 '기타손익비률',
 '금융손익비율',
 '금융비용부담률',
 '외환이익 대 매출액비율',
 '광고선전비 대 매출액비율',
 '세금과공과 대 세금과공과 차감전순이익률',
 '기업순이익률',
 '수지비율(관계기업투자손익 제외)',
 '인건비 대 총비용비율',
 'R & D 투자효율',
 '세금과공과 대 총비용비율',
 '금융비용 대 총비용비율',
 '감가상각비 대 총비용비율',
 '감가상각률',
 '누적감가상각률',
 '이자부담률',
 '지급이자율',
 '차입금평균이자율',
 '유보율',
 '적립금비율(재정비율)',
 '1주당매출액',
 '1주당순이익',
 '1주당 CASH FLOW',
 '1주당순자산',
 '1주당정상영업이익',
 '유동자산구성비율',
 '재고자산 대 유동자산비율',
 '유동자산 대 비유동자산비율',
 '당좌자산구성비율',
 '비유동자산구성비율',
 '자기자본구성비율',
 '타인자본구성비율',
 '자기자본배율',
 '비유동비율',
 '비유동장기적합률',
 '유동비율',
 '당좌비율',
 '현금비율',
 '매출채권비율',
 '재고자산 대 순운전자본비율',
 '매출채권 대 매입채무비율',
 '매입채무 대 재고자산비율',
 '부채비율',
 '유동부채비율',
 '단기차입금 대 총차입금비율',
 '비유동부채비율',
 '비유동부채 대 순운전자본비율',
 '순운전자본비율',
 '차입금의존도',
 '차입금비율',
 '유보액대비율',
 '유보액 대 납입자본배율',
 '유동자산집중도',
 '비유동자산집중도',
 '투자집중도',
 'CASH FLOW 대 부채비율',


In [ ]:
a = df_all.copy()

In [ ]:
a.drop(['상장협 산업분류(소분류)_y',
 '상장협 산업분류(중분류)_y',
 '자본(*)(IFRS)(천원)',
 '유형자산(*)(IFRS)(천원)',
 '비유동생물자산(*)(IFRS)(천원)',
 '투자부동산(*)(IFRS)(천원)',
 '비유동자산(*)(IFRS)(천원)',
 '유동자산(*)(IFRS)(천원)',
 '재고자산(*)(IFRS)(천원)',
 '* (정상)영업손익(계산수치)(IFRS)(천원)',
 '* (정상)영업손익(보고서기재)(IFRS)(천원)',
 '당기순이익(손실)(IFRS)(천원)',
 '총포괄손익(IFRS)(천원)',
 '매출액(수익)(*)(IFRS)(천원)',
 '부채(*)(IFRS)(천원)',
 '자산_y'],axis=1,inplace=True)

In [ ]:
a.shape

(4654, 129)

In [ ]:
a.describe()

,거래소코드,매출액총이익률,매출액정상영업이익률,매출액순이익률,총자본사업이익률,총자본정상영업이익률,총자본순이익률,자기자본정상영업이익률,자기자본순이익률,경영자본정상영업이익률,경영자본순이익률,자본금정상영업이익률,자본금순이익률,매출원가 대 매출액비율,영업비용 대 영업수익비율,기타손익비률,금융손익비율,금융비용부담률,외환이익 대 매출액비율,광고선전비 대 매출액비율,세금과공과 대 세금과공과 차감전순이익률,기업순이익률,수지비율(관계기업투자손익 제외),인건비 대 총비용비율,R & D 투자효율,세금과공과 대 총비용비율,금융비용 대 총비용비율,감가상각비 대 총비용비율,감가상각률,누적감가상각률,이자부담률,지급이자율,차입금평균이자율,유보율,적립금비율(재정비율),1주당매출액,1주당순이익,1주당 CASH FLOW,1주당순자산,1주당정상영업이익,유동자산구성비율,재고자산 대 유동자산비율,유동자산 대 비유동자산비율,당좌자산구성비율,비유동자산구성비율,자기자본구성비율,타인자본구성비율,자기자본배율,비유동비율,비유동장기적합률,유동비율,당좌비율,현금비율,매출채권비율,재고자산 대 순운전자본비율,매출채권 대 매입채무비율,매입채무 대 재고자산비율,부채비율,유동부채비율,단기차입금 대 총차입금비율,비유동부채비율,비유동부채 대 순운전자본비율,순운전자본비율,차입금의존도,차입금비율,유보액대비율,유보액 대 납입자본배율,유동자산집중도,비유동자산집중도,투자집중도,CASH FLOW 대 부채비율,CASH FLOW 대 차입금비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,정상영업이익대비이자보상배율,총자본회전률,경영자본회전률,자기자본회전률,자본금회전률,타인자본회전률,매입채무회전률,매입채무회전기간,유동자산회전률,당좌자산회전률,재고자산회전률,재고자산회전기간,재공품회전률,매출채권회전률,매출채권회전기간,비유동자산회전률,유형자산회전율,순운전자본회전률,운전자본회전률,1회전기간,부가가치(백만원),총자본투자효율,설비투자효율,기계투자효율,부가가치율,노동소득분배율,자본분배율,이윤분배율,영업활동으로 인한 현금흐름(간접법),투자활동으로 인한 현금흐름,재무활동으로 인한 현금흐름,유동부채,자산_x,자본,매출액(수익),매출액증가율,유형자산증가율,비유동생물자산증가율,투자부동산증가율,유동자산증가율,비유동자산증가율,재고자산증가율,정상영업이익증가율,순이익증가율,총포괄이익증가율,부실,총자본증가율,자기자본증가율
count,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4.654000e+03,4654.000000,4654.000000,12.00000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.00000,4.654000e+03,4654.000000,4.654000e+03,4.654000e+03,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4.654000e+03,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4.654000e+03,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4.654000e+03,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.00000,4.654000e+03,4654.000000,4654.000000,4.654000e+03,4654.000000,4654.000000,4654.000000,4654.000000,4.654000e+03,4.654000e+03,4.654000e+03,4.654000e+03,4.654000e+03,4.654000e+03,4.654000e+03,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000,4654.000000
mean,46399.746455,21.052396,-0.639366,-5.749454,3.105679,2.635440,-0.023651,4.443363,-26.994486,3.261919,-0.193025,223.834368,136.762576,73.554394,100.939955,-5.280026,-1.670892,1.652727,-0.017815,1.148790,32.159484,0.686650,102.744794,14.131594,0.859345,0.226465,1.007686,4.258977,14.452937,76.929540,0.466566,12.837048,12.357733,1939.128722,79.880548,7.327222e+04,1776.936657,3929.771304,102834.49750,3396.973176,46.483167,20.628539,134.604519,37.076386,53.335748,52.807643,47.324671,1950.645453,129.082428,85.185037,199.520247,165.841478,43.46167,7.983878e+03,155.288616,2.766095e+03,2.657950e+03,141.873915,100.049688,47.694875,40.862615,246.222241,12.401487,23.839538,76.408221,44.180290,24.321474,77.232486,101.234267,138.852566,11.703107,9.125976e+02,3.401184,1.530569,9.960675,0.923507,1.139843,2.153535,40.930604,2.360211,5.725059e+02,33.870447,2.174682,2.972441,417.711193,45.303427,7.787233e+03,10.689757,63.036448,2.265432,13.209820,1.171306,0.572935,0.04408,3.086556e+05,17.196437,234.993328,1.174242e+07,16.983853,103.368767,-13.274224,-66.101304,1.069380e+08,-1.147158e+08,1.211016e+07,6.391855e+08,2.104710e+09,8.467170e+08,1.571906e+09,0.203127,0.777743,0.007122,1.136667,0.145192,0.161827,7.454997,-0.699644,-0.604514,-0.562287,0.012677,0.104597,0.106767
std,48978.059903,18.668267,41.906462,92.756832,8.564196,8.536625,14.608465,20.778598,1483.70

In [ ]:
# a: 총자본증가율과 자기자본증가율 수정한 데이터프레임
# a.to_csv('./drive/MyDrive/공유문서함/상장csv/240316상장_비주력전처리중(1,2,3완료).csv')

#### 4. 결측치 확인 및 이상치 확인

##### 주력산업

In [ ]:
major3 = pd.read_csv('./drive/MyDrive/공유문서함/상장csv/240316상장_주력전처리중(1,2,3완료).csv', index_col=0)

In [ ]:
df = pd.read_csv('./drive/MyDrive/공유문서함/상장csv/240315상장_주력산업(new).csv', index_col=0)

In [ ]:
major3.shape

(6507, 129)

In [ ]:
major3['부실'].value_counts()

0.0    6400
1.0     107
Name: 부실, dtype: int64

In [ ]:
# column별 결측치 개수 확인
nan_table = pd.DataFrame()

def find_nan(df):
  for column in df.columns:
    if df[column].isna().sum() != 0:
      print(column, df[column].isna().sum())
     # x = df[df[column].isna()]['회사명'].values
     # print(x)

  return nan_table

In [ ]:
find_nan(major3)

1주당순자산 6499
상장폐지일 6404


""


In [ ]:
major3.describe().round(3)

,거래소코드,매출액총이익률,매출액정상영업이익률,매출액순이익률,총자본사업이익률,총자본정상영업이익률,총자본순이익률,자기자본정상영업이익률,자기자본순이익률,경영자본정상영업이익률,경영자본순이익률,자본금정상영업이익률,자본금순이익률,매출원가 대 매출액비율,영업비용 대 영업수익비율,기타손익비률,금융손익비율,금융비용부담률,외환이익 대 매출액비율,광고선전비 대 매출액비율,세금과공과 대 세금과공과 차감전순이익률,기업순이익률,수지비율(관계기업투자손익 제외),인건비 대 총비용비율,R & D 투자효율,세금과공과 대 총비용비율,금융비용 대 총비용비율,감가상각비 대 총비용비율,감가상각률,누적감가상각률,이자부담률,지급이자율,차입금평균이자율,유보율,적립금비율(재정비율),1주당매출액,1주당순이익,1주당 CASH FLOW,1주당정상영업이익,유동자산구성비율,재고자산 대 유동자산비율,유동자산 대 비유동자산비율,당좌자산구성비율,비유동자산구성비율,자기자본구성비율,타인자본구성비율,자기자본배율,비유동비율,비유동장기적합률,유동비율,당좌비율,현금비율,매출채권비율,재고자산 대 순운전자본비율,매출채권 대 매입채무비율,매입채무 대 재고자산비율,부채비율,유동부채비율,단기차입금 대 총차입금비율,비유동부채비율,비유동부채 대 순운전자본비율,순운전자본비율,차입금의존도,차입금비율,유보액대비율,유보액 대 납입자본배율,유동자산집중도,비유동자산집중도,투자집중도,CASH FLOW 대 부채비율,CASH FLOW 대 차입금비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,정상영업이익대비이자보상배율,총자본회전률,경영자본회전률,자기자본회전률,자본금회전률,타인자본회전률,매입채무회전률,매입채무회전기간,유동자산회전률,당좌자산회전률,재고자산회전률,재고자산회전기간,재공품회전률,매출채권회전률,매출채권회전기간,비유동자산회전률,유형자산회전율,순운전자본회전률,운전자본회전률,1회전기간,부가가치(백만원),총자본투자효율,설비투자효율,기계투자효율,부가가치율,노동소득분배율,자본분배율,이윤분배율,영업활동으로 인한 현금흐름(간접법),투자활동으로 인한 현금흐름,재무활동으로 인한 현금흐름,유동부채,자산_x,자본,매출액(수익),매출액증가율,유형자산증가율,비유동생물자산증가율,투자부동산증가율,유동자산증가율,비유동자산증가율,재고자산증가율,정상영업이익증가율,순이익증가율,총포괄이익증가율,부실,총자본증가율,자기자본증가율
count,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6.507000e+03,6507.000,6507.000,6.507000e+03,6507.000,6507.000,6507.000,6507.000,6.507000e+03,6.507000e+03,6.507000e+03,6.507000e+03,6.507000e+03,6.507000e+03,6.507000e+03,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000,6507.000
mean,61802.523,21.016,0.677,-4.254,3.621,3.198,0.511,4.777,-3.501,3.587,0.268,146.787,88.146,76.648,99.533,-3.638,-1.436,1.096,-0.005,0.806,14.762,1.210,100.742,15.222,1.146,0.120,0.799,4.285,13.608,82.851,0.199,7.296,5.744,1517.110,77.056,39736.821,1491.508,3067.782,2329.434,51.066,24.442,143.881,39.142,48.690,55.602,44.490,1562.868,127.809,82.280,248.885,200.401,53.842,1435.475,291.025,412.872,727.491,148.386,105.335,51.302,40.793,535.942,17.381,23.556,85.172,48.178,16.295,76.200,78.610,129.290,16.948,1235.629,4.202,3.328,19.956,0.915,1.057,2.016,25.971,2.625,24.332,35.971,1.880,2.657,84.069,55.863,1488.754,7.457,73.225,2.288,6.795,1.095,1.518,0.024,3.414235e+05,17.332,144.800,2.006104e+05,19.615,93.598,-0.837,-38.915,1.519755e+08,-1.448830e+08,1.330935e+06,5.076727e+08,1.876604e+09,9.413415e+08,1.537311e+09,0.090,0.658,0.000,0.338,0.092,0.145,0.447,-0.330,-5.498,-0.470,0.016,0.084,0.109
std,54716.997,18.282,59.988,69.913,8.621,8.598,13.232,22.422,65.487,11.437,19.255,397.947,357.256,21.546,60.024,29.899,14.234,5.710,1.422,2.878,473.889,12.698,34.944,9.617,8.243,0.411,2.252,3.268,6.141,79.962,5.647,54.726,41.126,3096.814,162.436,152360.569,11038.453,19546.281,13400.931,15.943,15.082,280.951,16.328,15.894,26.857,27.151,2933.346,848.281,73.663,345.703,325.083,110.000,26126.923,9058.692,1739.715,14057.417,1199.424,520.077,35.523,729.192,25964.979,29.477,19.975,883.843,32.249,34.085,180.181,141.869,247.824,53.188,41767.317,7.390,41.809,699.291,0.472,0.519,1.935,39.576,1.912,239.857,32.529,0.958,1.572,1933.720,51.681,54878.077,42.897,53.783,2.081,20.055,0.498,102.366,0.132,3.204405e+06,17.595,833.554,3.590720e+07,43.911,991.528,991.183,1117.272,1.834536e+09,1.528087e+09,4.107101e+08,3.002917e+09,

In [ ]:
# 6499 결측치인 1주당순자산 제거
major3.drop('1주당순자산', axis=1,inplace=True)

In [ ]:
find_nan(major3)

상장폐지일 6404


""


##### 증가율 피처 중 최소값이 -100이하인 경우 확인 및 처리
  - 정상영업이익증가율, 순이익증가율, 총포괄이익증가율은 영업이익(손실), 당기순이익(손실), 총포괄손익으로 계산되기 때문에 가능

In [ ]:
major3.sort_values('정상영업이익증가율', ascending=True)

,회사명,거래소코드,회계년도,매출액총이익률,매출액정상영업이익률,매출액순이익률,총자본사업이익률,총자본정상영업이익률,총자본순이익률,자기자본정상영업이익률,자기자본순이익률,경영자본정상영업이익률,경영자본순이익률,자본금정상영업이익률,자본금순이익률,매출원가 대 매출액비율,영업비용 대 영업수익비율,기타손익비률,금융손익비율,금융비용부담률,외환이익 대 매출액비율,광고선전비 대 매출액비율,세금과공과 대 세금과공과 차감전순이익률,기업순이익률,수지비율(관계기업투자손익 제외),인건비 대 총비용비율,R & D 투자효율,세금과공과 대 총비용비율,금융비용 대 총비용비율,감가상각비 대 총비용비율,감가상각률,누적감가상각률,이자부담률,지급이자율,차입금평균이자율,유보율,적립금비율(재정비율),1주당매출액,1주당순이익,1주당 CASH FLOW,1주당정상영업이익,유동자산구성비율,재고자산 대 유동자산비율,유동자산 대 비유동자산비율,당좌자산구성비율,비유동자산구성비율,자기자본구성비율,타인자본구성비율,자기자본배율,비유동비율,비유동장기적합률,유동비율,당좌비율,현금비율,매출채권비율,재고자산 대 순운전자본비율,매출채권 대 매입채무비율,매입채무 대 재고자산비율,부채비율,유동부채비율,단기차입금 대 총차입금비율,비유동부채비율,비유동부채 대 순운전자본비율,순운전자본비율,차입금의존도,차입금비율,유보액대비율,유보액 대 납입자본배율,유동자산집중도,비유동자산집중도,투자집중도,CASH FLOW 대 부채비율,CASH FLOW 대 차입금비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,정상영업이익대비이자보상배율,총자본회전률,경영자본회전률,자기자본회전률,자본금회전률,타인자본회전률,매입채무회전률,매입채무회전기간,유동자산회전률,당좌자산회전률,재고자산회전률,재고자산회전기간,재공품회전률,매출채권회전률,매출채권회전기간,비유동자산회전률,유형자산회전율,순운전자본회전률,운전자본회전률,1회전기간,부가가치(백만원),총자본투자효율,설비투자효율,기계투자효율,부가가치율,노동소득분배율,자본분배율,이윤분배율,설립일,상장일,상장폐지일,영업활동으로 인한 현금흐름(간접법),투자활동으로 인한 현금흐름,재무활동으로 인한 현금흐름,유동부채,자산_x,자본,매출액(수익),상장협 산업분류(소분류)_x,상장협 산업분류(중분류)_x,매출액증가율,유형자산증가율,비유동생물자산증가율,투자부동산증가율,유동자산증가율,비유동자산증가율,재고자산증가율,정상영업이익증가율,순이익증가율,총포괄이익증가율,부실,총자본증가율,자기자본증가율
4187,엔시트론,101400,2017-12-01,15.11,-22.15,-33.26,-15.34,-15.60,-23.42,-41.57,-62.43,-22.63,-33.98,-77.54,-116.45,84.89,122.15,-6.07,-7.87,5.59,-3.07,0.14,0.00,-19.49,126.39,9.47,3.29,0.00,4.13,7.22,30.80,62.57,6.30,7.21,7.99,106.39,56.86,1867.61,-120.0,-417.93,-413.67,49.29,34.00,97.19,32.53,50.71,39.40,60.60,194.04,128.71,76.66,145.63,96.11,4.73,53.47,108.52,295.93,18.07,153.80,85.90,52.33,67.90,173.23,15.44,54.24,137.65,21.60,1.06,76.20,65.81,95.13,-12.81,-14.31,-7.76,-11.82,-3.96,0.70,1.02,1.88,3.50,1.13,10.65,34.26,1.31,2.00,3.82,95.65,34.59,4.84,75.39,1.52,11.82,0.76,0.00,0.01,-20.49,-0.05,-1.00,-3.86,-0.08,0.00,0.00,0.00,2000/04/04,2009-03-31,NaN,-5874798.0,-10319257.0,15462570.0,1.352799e+07,3.997140e+07,1.574932e+07,2.625273e+07,반도체 제조업,"전자부품, 컴퓨터, 영상, 음향 및 통신장비 제조업",-0.371769,-0.104280,0.0,-1.000000,-0.029843,0.419100,-0.051013,-1903.171737,0.000000,0.000000,0.0,0.155542,0.288239
225,금빛,45890,2018-12-01,5.24,-31.73,-12.50,-15.60,-15.90,-6.26,-35.63,-14.04,-16.71,-6.58,-127.46,-50.20,94.76,131.73,15.45,-2.54,3.00,2.03,0.42,0.00,-4.76,112.15,17.92,0.00,0.00,2.00,3.75,10.07,56.27,2.71,4.90,4.56,351.60,64.98,1999.62,-256.0,-63.84,-634.51,43.43,35.62,76.76,27.96,56.57,48.56,51.44,374.84,116.50,92.33,112.13,72.19,10.50,71.82,329.19,152.85,46.99,105.94,79.76,74.35,26.18,270.55,4.70,38.00,78.25,45.55,3.52,92.36,107.26,190.59,-4.31,-5.84,-2.22,-4.28,-10.57,0.50,0.53,1.12,4.02,0.90,5.33,68.47,1.08,1.72,2.92,124.90,16.85,3.37,108.18,0.93,1.12,0.60,1.04,0.01,10254.87,9.99,18.70,595.25,19.29,139.63,-39.63,-64.80,1999/07/15,2005-11-28,NaN,-1697350.0,7775765.0,-5865181.0,3.974045e+07,1.026127e+08,4.982730e+07,5.316230e+07,전구 및 조명장치 제조업,전기장비 제조업,-0.270794,0.378346,0.0,0.000000,-0.169221,0.036653,-0.226249,-286.316279,0.000000,0.000000,0.0,-0.064069,0.110786
5753,인텍플러스,64290,2018-12-01,12.63,-51.64,-40.42,-26.40,-26.74,-20.93,-47.71,-36.80,-27.34,-21.40,-141.20,-108.42,87.37,151.64,11.22,-0.10,0.88,0.82,0.00,0.00,-20.47,134.89,40.47,0.00,1.57,0.56,1.81,7.36,53.14,1.03,3.60,4.02,187.48,71.85,1458.70,-600.0,-519.09,-753.34,58.96,48.77,143.67,30.21,41.04,54.46,45.54,262.25,75.36,64.84,160.62,82.29,36.68,13.37,129.21,48.77,27.41,83.62,67.40,80.38,16.22,39.69,22.25,22.45,41.23,39.01,1.89,117.81,75.35,183.28,-16.80,-34.07,-7.65,-14.39,-58.89,0.52,0.53,0.92,2.73,1.18,6.53,55.89,0.85,1.53,1.91,190.77,3.44,5.00,73.05,1.33,1.69,0.53,2.64,0.00,4668.10,15.40,49.80,1161.82,28.98,218.23,-118.23,-139.49,1995/10/13,2011-01-05,NaN,-5732524.0,-2593782.0,7034089.0,1.112518e+07,3.030781e+07,1.650583e+07,1.610910e+07,특수 목적용 기계 제조업,기타 기계 및 장비 제조업,-0.337226,-0.033173,0.0,0.000000,-0.110413,0.050563,0.072595,-285.727780,0.000000,0.000000,0.0,-0.050718,-0.101532
1229,씨에스에이코스믹,83660,2019-12-01,42.86,-8.91,-11.88,-16.07,-16.21,-21.61,-28.54,-38.05,-16.65,-22.20,-58.01,-77.34,57.14,10

  영업이익(손실)항목으로 계산되어서 -100이하의 값이 나옴

In [ ]:
major3.sort_values('순이익증가율', ascending=True)

,회사명,거래소코드,회계년도,매출액총이익률,매출액정상영업이익률,매출액순이익률,총자본사업이익률,총자본정상영업이익률,총자본순이익률,자기자본정상영업이익률,자기자본순이익률,경영자본정상영업이익률,경영자본순이익률,자본금정상영업이익률,자본금순이익률,매출원가 대 매출액비율,영업비용 대 영업수익비율,기타손익비률,금융손익비율,금융비용부담률,외환이익 대 매출액비율,광고선전비 대 매출액비율,세금과공과 대 세금과공과 차감전순이익률,기업순이익률,수지비율(관계기업투자손익 제외),인건비 대 총비용비율,R & D 투자효율,세금과공과 대 총비용비율,금융비용 대 총비용비율,감가상각비 대 총비용비율,감가상각률,누적감가상각률,이자부담률,지급이자율,차입금평균이자율,유보율,적립금비율(재정비율),1주당매출액,1주당순이익,1주당 CASH FLOW,1주당정상영업이익,유동자산구성비율,재고자산 대 유동자산비율,유동자산 대 비유동자산비율,당좌자산구성비율,비유동자산구성비율,자기자본구성비율,타인자본구성비율,자기자본배율,비유동비율,비유동장기적합률,유동비율,당좌비율,현금비율,매출채권비율,재고자산 대 순운전자본비율,매출채권 대 매입채무비율,매입채무 대 재고자산비율,부채비율,유동부채비율,단기차입금 대 총차입금비율,비유동부채비율,비유동부채 대 순운전자본비율,순운전자본비율,차입금의존도,차입금비율,유보액대비율,유보액 대 납입자본배율,유동자산집중도,비유동자산집중도,투자집중도,CASH FLOW 대 부채비율,CASH FLOW 대 차입금비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,정상영업이익대비이자보상배율,총자본회전률,경영자본회전률,자기자본회전률,자본금회전률,타인자본회전률,매입채무회전률,매입채무회전기간,유동자산회전률,당좌자산회전률,재고자산회전률,재고자산회전기간,재공품회전률,매출채권회전률,매출채권회전기간,비유동자산회전률,유형자산회전율,순운전자본회전률,운전자본회전률,1회전기간,부가가치(백만원),총자본투자효율,설비투자효율,기계투자효율,부가가치율,노동소득분배율,자본분배율,이윤분배율,설립일,상장일,상장폐지일,영업활동으로 인한 현금흐름(간접법),투자활동으로 인한 현금흐름,재무활동으로 인한 현금흐름,유동부채,자산_x,자본,매출액(수익),상장협 산업분류(소분류)_x,상장협 산업분류(중분류)_x,매출액증가율,유형자산증가율,비유동생물자산증가율,투자부동산증가율,유동자산증가율,비유동자산증가율,재고자산증가율,정상영업이익증가율,순이익증가율,총포괄이익증가율,부실,총자본증가율,자기자본증가율
5077,현대정보기술,26180,2012-12-01,0.32,-2.65,-3.37,-3.03,-3.22,-4.09,-9.85,-12.54,-4.19,-5.33,-9.98,-12.70,99.68,102.65,-0.72,-1.15,1.38,-0.15,0.00,0.00,-2.42,103.33,17.66,0.00,0.00,1.32,5.90,16.24,93.73,2.49,5.23,5.23,1.34,3.03,3764.51,-127.0,-74.91,-99.78,24.55,0.35,32.54,24.47,75.45,32.66,67.34,101.30,231.01,160.68,46.28,46.12,1.04,10313.32,0.00,81.89,12593.78,206.18,162.41,99.91,43.77,0.00,-28.49,35.60,109.01,0.44,0.01,20.23,62.17,63.29,-3.59,-6.78,-2.41,-1.99,-1.92,1.21,1.58,3.72,3.76,1.80,11.36,32.12,4.94,4.96,1431.23,0.26,0.00,13.88,26.30,1.61,2.50,1.56,0.00,0.00,44430.29,28.46,58.52,198.60,23.45,78.76,21.24,-14.39,1989/05/15,2000-08-08,NaN,-3770361.0,951870.0,2980970.0,82820873.0,1.561355e+08,50994871.0,1.894803e+08,"컴퓨터 프로그래밍, 시스템 통합 및 관리업","컴퓨터 프로그래밍, 시스템 통합 및 관리업",0.015455,-0.299699,0.0,0.000000,-0.199155,-0.028765,-0.403073,-2.620620,-30435.923810,0.000000,0.0,-0.076977,-0.128410
380,다산솔루에타,154040,2018-12-01,13.87,2.43,-7.23,3.04,2.67,-7.94,7.15,-20.19,2.99,-8.88,95.72,-236.89,86.13,97.57,-9.28,-1.64,1.54,0.07,0.00,0.00,-6.24,105.64,11.10,0.00,0.14,1.39,3.20,13.21,83.45,2.70,7.43,7.58,940.75,95.26,18805.73,-1147.0,254.10,457.63,54.55,45.74,120.02,29.60,45.45,33.64,66.36,1040.75,135.11,109.20,93.44,50.70,10.81,67.00,0.00,101.74,65.85,197.28,173.56,80.42,23.73,0.00,-3.83,27.00,80.26,30.79,10.80,48.60,42.46,80.67,0.73,1.79,0.48,0.45,1.58,1.10,1.23,2.94,39.34,1.75,6.79,53.77,2.06,2.92,6.97,52.39,10.45,5.44,67.06,2.36,4.00,1.06,0.18,0.02,24285.15,11.45,46.24,149.97,10.68,115.00,-15.00,-67.64,2003/06/27,2013-12-27,NaN,3071204.0,-12017399.0,-1026266.0,123798560.0,2.120544e+08,71330869.0,2.272971e+08,전자부품 제조업,"전자부품, 컴퓨터, 영상, 음향 및 통신장비 제조업",0.457706,-0.086729,0.0,-0.009594,0.099041,-0.002719,3.287590,0.955639,-1826.367819,0.000000,0.0,0.050331,-0.144183
1232,씨엔플러스,115530,2014-12-01,1.29,-20.79,-38.23,-11.92,-12.14,-22.33,-21.92,-40.21,-15.00,-27.58,-301.38,-551.32,98.71,120.79,-17.43,-0.81,1.25,-1.43,0.00,0.00,-21.60,136.91,16.94,10.58,0.00,0.89,11.00,18.39,75.98,1.63,2.17,2.78,1038.85,93.95,7081.77,-2822.0,-338.44,-1472.32,26.03,18.40,35.19,21.24,73.97,45.43,54.57,1100.45,162.83,126.35,62.79,51.23,7.35,171.50,0.00,195.65,87.65,120.12,91.25,75.42,28.87,0.00,-15.43,44.34,97.61,42.91,10.47,45.77,125.45,133.09,1.96,2.41,1.07,1.80,-16.69,0.58,0.72,1.05,14.50,1.31,10.72,34.05,2.18,2.58,14.24,25.63,40.29,5.74,63.61,0.80,0.98,0.96,0.11,0.02,764.27,1.33,2.89,7.60,2.23,1066.16,-966.16,-1712.39,2003/11/10,2011-11-02,NaN,-1636029.0,-8612337.0,10760882.0,23889917.0,5.762842e+07,26180272.0,3.423327e+07,전자부품 제조업,"전자부품, 컴퓨터, 영상, 음향 및 통신장비 제조업",-0.198651,0.028783,0.0,0.000000,-0.081820,-0.014670,0.348639,0.000000,-362.068670,-160.434630,0.0,-0.033077,-0.324680
6399,파나케이아,58530,2017-12-01,13.26,-14.39,-24.58,-12.04,-12.29,-21.00,-28.34,-42.36,-13.45,-22.99,-62.33,-89

당기순이익(손실)항목으로 손실도 기재되기 때문에 -100이하값 있음

총포괄손익항목으로 계산되기 때문에 -100이하 가능

##### 피처의 밸류값이 대부분이 0인 컬럼 확인 및 제거
  - 세금과공과 대 총비용비율,R & D 투자효율, 비유동생물자산증가율 제거

In [ ]:
major3.describe()

,거래소코드,매출액총이익률,매출액정상영업이익률,매출액순이익률,총자본사업이익률,총자본정상영업이익률,총자본순이익률,자기자본정상영업이익률,자기자본순이익률,경영자본정상영업이익률,경영자본순이익률,자본금정상영업이익률,자본금순이익률,매출원가 대 매출액비율,영업비용 대 영업수익비율,기타손익비률,금융손익비율,금융비용부담률,외환이익 대 매출액비율,광고선전비 대 매출액비율,세금과공과 대 세금과공과 차감전순이익률,기업순이익률,수지비율(관계기업투자손익 제외),인건비 대 총비용비율,R & D 투자효율,세금과공과 대 총비용비율,금융비용 대 총비용비율,감가상각비 대 총비용비율,감가상각률,누적감가상각률,이자부담률,지급이자율,차입금평균이자율,유보율,적립금비율(재정비율),1주당매출액,1주당순이익,1주당 CASH FLOW,1주당정상영업이익,유동자산구성비율,재고자산 대 유동자산비율,유동자산 대 비유동자산비율,당좌자산구성비율,비유동자산구성비율,자기자본구성비율,타인자본구성비율,자기자본배율,비유동비율,비유동장기적합률,유동비율,당좌비율,현금비율,매출채권비율,재고자산 대 순운전자본비율,매출채권 대 매입채무비율,매입채무 대 재고자산비율,부채비율,유동부채비율,단기차입금 대 총차입금비율,비유동부채비율,비유동부채 대 순운전자본비율,순운전자본비율,차입금의존도,차입금비율,유보액대비율,유보액 대 납입자본배율,유동자산집중도,비유동자산집중도,투자집중도,CASH FLOW 대 부채비율,CASH FLOW 대 차입금비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,정상영업이익대비이자보상배율,총자본회전률,경영자본회전률,자기자본회전률,자본금회전률,타인자본회전률,매입채무회전률,매입채무회전기간,유동자산회전률,당좌자산회전률,재고자산회전률,재고자산회전기간,재공품회전률,매출채권회전률,매출채권회전기간,비유동자산회전률,유형자산회전율,순운전자본회전률,운전자본회전률,1회전기간,부가가치(백만원),총자본투자효율,설비투자효율,기계투자효율,부가가치율,노동소득분배율,자본분배율,이윤분배율,영업활동으로 인한 현금흐름(간접법),투자활동으로 인한 현금흐름,재무활동으로 인한 현금흐름,유동부채,자산_x,자본,매출액(수익),매출액증가율,유형자산증가율,비유동생물자산증가율,투자부동산증가율,유동자산증가율,비유동자산증가율,재고자산증가율,정상영업이익증가율,순이익증가율,총포괄이익증가율,부실,총자본증가율,자기자본증가율
count,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6.507000e+03,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6.507000e+03,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6.507000e+03,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6.507000e+03,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6.507000e+03,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6.507000e+03,6507.000000,6507.000000,6.507000e+03,6507.000000,6507.000000,6507.000000,6507.000000,6.507000e+03,6.507000e+03,6.507000e+03,6.507000e+03,6.507000e+03,6.507000e+03,6.507000e+03,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000
mean,61802.523436,21.015771,0.676819,-4.253988,3.621497,3.198285,0.511105,4.777143,-3.501411,3.586933,0.268416,146.786501,88.145880,76.648283,99.532925,-3.637501,-1.435585,1.095811,-0.004646,0.805751,14.761821,1.210410,100.742264,15.221634,1.146321,0.119720,0.799119,4.284501,13.607802,82.851030,0.199099,7.295889,5.744140,1517.109588,77.056087,3.973682e+04,1491.508117,3067.781843,2329.433575,51.065912,24.441830,143.880996,39.142305,48.690092,55.602313,44.490148,1562.868419,127.808606,82.280272,248.884836,200.401252,53.842004,1.435475e+03,291.024640,412.871924,727.490602,148.386113,105.335002,51.302170,40.793275,5.359418e+02,17.381405,23.555863,85.171903,48.178139,16.294872,76.200314,78.610438,129.290269,16.948463,1.235629e+03,4.201777,3.328019,19.956495,0.914689,1.057470,2.016003,25.970899,2.625216,24.332110,35.970589,1.879596,2.657191,84.069333,55.863015,1.488754e+03,7.457495,73.225105,2.287993,6.794846,1.095494,1.517872,0.024289,3.414235e+05,17.331654,144.800092,2.006104e+05,19.614621,93.598156,-0.836515,-38.914627,1.519755e+08,-1.448830e+08,1.330935e+06,5.076727e+08,1.876604e+09,9.413415e+08,1.537311e+09,0.090295,0.658372,0.000429,0.337514,0.091702,0.144585,0.446908,-0.329679,-5.497993,-0.469576,0.016444,0.083767,0.109406
std,54716.997323,18.282104,59.988057,69.913204,8.620701,8.598316,13.232408,22.422076,65.486517,11.436553,19.255251,397.946592,357.255573,

In [ ]:
major3[['세금과공과 대 총비용비율']].value_counts() # 주력산업 총 6507 중 5434개가 0값

세금과공과 대 총비용비율
0.00             5434
0.36               23
0.38               20
0.50               18
0.49               18
                 ... 
1.66                1
1.65                1
1.64                1
1.61                1
9.02                1
Length: 221, dtype: int64

In [ ]:
major3[['R & D 투자효율']].value_counts()

R & D 투자효율
0.00          4866
0.37            11
0.91            10
2.28            10
2.75             9
              ... 
5.55             1
5.54             1
5.53             1
5.49             1
481.00           1
Length: 754, dtype: int64

In [ ]:
major3[['비유동생물자산증가율']].value_counts() # 주력산업 총 6507 중 6495개가 0값

비유동생물자산증가율
 0.000000     6495
-1.000000        1
-0.276037        1
-0.009535        1
 0.083929        1
 0.097336        1
 0.113896        1
 0.114600        1
 0.173566        1
 0.249357        1
 0.428130        1
 1.232664        1
 1.580994        1
dtype: int64

In [ ]:
major3.drop(['세금과공과 대 총비용비율','R & D 투자효율', '비유동생물자산증가율'],axis=1, inplace=True)

##### 비주력산업

In [ ]:
minor3 =pd.read_csv('./drive/MyDrive/공유문서함/상장csv/240316상장_비주력전처리중(1,2,3완료).csv', index_col=0)

In [ ]:
minor3.head(3)

,회사명,거래소코드,회계년도,매출액총이익률,매출액정상영업이익률,매출액순이익률,총자본사업이익률,총자본정상영업이익률,총자본순이익률,자기자본정상영업이익률,자기자본순이익률,경영자본정상영업이익률,경영자본순이익률,자본금정상영업이익률,자본금순이익률,매출원가 대 매출액비율,영업비용 대 영업수익비율,기타손익비률,금융손익비율,금융비용부담률,외환이익 대 매출액비율,광고선전비 대 매출액비율,세금과공과 대 세금과공과 차감전순이익률,기업순이익률,수지비율(관계기업투자손익 제외),인건비 대 총비용비율,R & D 투자효율,세금과공과 대 총비용비율,금융비용 대 총비용비율,감가상각비 대 총비용비율,감가상각률,누적감가상각률,이자부담률,지급이자율,차입금평균이자율,유보율,적립금비율(재정비율),1주당매출액,1주당순이익,1주당 CASH FLOW,1주당순자산,1주당정상영업이익,유동자산구성비율,재고자산 대 유동자산비율,유동자산 대 비유동자산비율,당좌자산구성비율,비유동자산구성비율,자기자본구성비율,타인자본구성비율,자기자본배율,비유동비율,비유동장기적합률,유동비율,당좌비율,현금비율,매출채권비율,재고자산 대 순운전자본비율,매출채권 대 매입채무비율,매입채무 대 재고자산비율,부채비율,유동부채비율,단기차입금 대 총차입금비율,비유동부채비율,비유동부채 대 순운전자본비율,순운전자본비율,차입금의존도,차입금비율,유보액대비율,유보액 대 납입자본배율,유동자산집중도,비유동자산집중도,투자집중도,CASH FLOW 대 부채비율,CASH FLOW 대 차입금비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,정상영업이익대비이자보상배율,총자본회전률,경영자본회전률,자기자본회전률,자본금회전률,타인자본회전률,매입채무회전률,매입채무회전기간,유동자산회전률,당좌자산회전률,재고자산회전률,재고자산회전기간,재공품회전률,매출채권회전률,매출채권회전기간,비유동자산회전률,유형자산회전율,순운전자본회전률,운전자본회전률,1회전기간,부가가치(백만원),총자본투자효율,설비투자효율,기계투자효율,부가가치율,노동소득분배율,자본분배율,이윤분배율,설립일,상장일,상장폐지일,영업활동으로 인한 현금흐름(간접법),투자활동으로 인한 현금흐름,재무활동으로 인한 현금흐름,유동부채,자산_x,자본,매출액(수익),상장협 산업분류(소분류)_x,상장협 산업분류(중분류)_x,매출액증가율,유형자산증가율,비유동생물자산증가율,투자부동산증가율,유동자산증가율,비유동자산증가율,재고자산증가율,정상영업이익증가율,순이익증가율,총포괄이익증가율,부실,총자본증가율,자기자본증가율
0,HJ 매그놀리아 용평 호텔 앤 리조트,70960,2017-12-01,22.57,15.98,8.09,3.77,3.73,1.89,8.33,4.21,3.76,1.90,13.92,7.04,77.43,84.02,-5.53,-5.12,5.12,-0.04,0.86,37.47,3.08,89.61,11.95,0.0,2.76,5.69,7.86,5.49,78.63,2.16,7.86,8.22,69.59,41.04,4356.18,352.0,370.00,8146.08,695.92,16.43,9.85,19.65,14.81,83.57,47.02,52.98,169.59,177.75,118.80,55.39,49.94,12.62,631.34,0.0,5053.47,12.49,112.69,63.07,85.26,49.62,0.0,-13.23,16.35,34.78,19.30,0.70,90.51,338.03,412.40,-0.13,-0.42,-0.07,-0.28,3.12,0.23,0.24,0.52,0.87,0.42,112.74,3.24,1.10,1.19,14.85,24.59,0.0,1.84,198.51,0.30,0.30,0.44,0.0,0.00,77704.69,8.95,10.95,0.0,37.06,29.06,70.94,21.82,2000/02/07,2016-05-27,NaN,17809537.0,-61501445.0,1684160.0,257429706.0,868156494.0,408181173.0,209677450.0,일반 및 생활 숙박시설 운영업,숙박업,-0.004863,0.039747,0.0,0.0,-0.398240,0.048530,-0.011091,0.102818,0.183780,0.182663,0.0,-0.065439,0.029611
1,HJ 매그놀리아 용평 호텔 앤 리조트,70960,2018-12-01,22.91,13.54,5.62,2.97,2.92,1.21,5.97,2.48,2.93,1.22,10.21,4.23,77.09,86.46,-6.32,-6.54,5.59,0.29,0.62,46.60,2.42,92.82,15.16,0.0,3.53,5.98,9.85,5.91,85.50,2.36,6.80,7.08,72.09,41.67,3768.46,212.0,1090.82,8287.53,510.31,12.80,12.88,14.67,11.15,87.20,50.96,49.04,172.09,171.13,113.29,55.57,48.41,31.88,161.78,0.0,1474.81,10.97,96.24,45.19,75.68,51.06,0.0,-10.23,18.97,37.23,21.35,0.72,67.97,395.38,446.72,8.82,22.80,4.33,19.38,2.42,0.22,0.22,0.44,0.75,0.42,112.52,3.24,1.47,1.66,13.22,27.61,0.0,3.29,111.02,0.25,0.26,0.52,0.0,0.01,74163.43,9.12,10.70,0.0,40.89,34.64,65.36,13.73,2000/02/07,2016-05-27,NaN,52504906.0,-17193893.0,-8132498.0,187144500.0,812775928.0,414170559.0,181388591.0,일반 및 생활 숙박시설 운영업,숙박업,-0.134916,-0.026166,0.0,0.0,-0.270702,-0.023125,-0.046241,-0.266717,-0.399175,-0.356503,0.0,-0.063791,0.014673
2,HJ 매그놀리아 용평 호텔 앤 리조트,70960,2019-12-01,10.26,-3.19,-10.66,-0.41,-0.50,-1.66,-1.02,-3.41,-0.50,-1.67,-1.71,-5.73,89.74,103.19,-10.31,-8.24,9.57,-0.91,0.77,0.00,-0.17,113.18,17.47,0.0,3.48,8.26,11.80,6.10,82.36,2.91,7.24,7.81,64.45,39.13,2689.02,-287.0,490.32,7979.61,-85.68,12.80,14.66,14.68,10.92,87.20,46.57,53.43,164.45,187.26,118.27,48.72,41.58,22.05,131.24,0.0,1205.05,10.89,114.75,56.42,64.88,58.33,0.0,-13.47,21.26,45.66,18.25,0.64,82.20,560.11,650.91,8.38,21.06,4.48,29.40,-0.33,0.16,0.16,0.32,0.54,0.30,80.72,4.52,1.22,1.41,8.82,41.38,0.0,6.08,60.08,0.18,0.18,0.42,0.0,0.01,45500.21,5.35,6.32,0.0,35.15,57.59,42.41,-30.31,2000/02/07,2016-05-27,NaN,23600746.0,-20538858.0,-13495542.0,223287000.0,849935619.0,395782278.0,129431260.0,일반 및 생활 숙박시설 운영업,숙박업,-0.286442,0.048206,0.0,0.0,0.046091,0.045665,0.190960,-1.167902,-2.353979,-2.409036,0.0,0.045719,-0.044398


In [ ]:
minor3.shape

(4654, 129)

In [ ]:
minor3['부실'].value_counts()

0.0    4595
1.0      59
Name: 부실, dtype: int64

In [ ]:
# 결측치 확인
find_nan(minor3)

1주당순자산 4642
상장폐지일 4595


""


In [ ]:
# 총 row 4595 중 4642가 결측치인 1주당순자산 컬럼 제거
minor3.drop('1주당순자산',axis=1, inplace=True)

##### 증가율 피처 중 최소값이 -100이하인 경우 확인 및 처리

In [ ]:
minor3.describe().round(3)

,거래소코드,매출액총이익률,매출액정상영업이익률,매출액순이익률,총자본사업이익률,총자본정상영업이익률,총자본순이익률,자기자본정상영업이익률,자기자본순이익률,경영자본정상영업이익률,경영자본순이익률,자본금정상영업이익률,자본금순이익률,매출원가 대 매출액비율,영업비용 대 영업수익비율,기타손익비률,금융손익비율,금융비용부담률,외환이익 대 매출액비율,광고선전비 대 매출액비율,세금과공과 대 세금과공과 차감전순이익률,기업순이익률,수지비율(관계기업투자손익 제외),인건비 대 총비용비율,R & D 투자효율,세금과공과 대 총비용비율,금융비용 대 총비용비율,감가상각비 대 총비용비율,감가상각률,누적감가상각률,이자부담률,지급이자율,차입금평균이자율,유보율,적립금비율(재정비율),1주당매출액,1주당순이익,1주당 CASH FLOW,1주당정상영업이익,유동자산구성비율,재고자산 대 유동자산비율,유동자산 대 비유동자산비율,당좌자산구성비율,비유동자산구성비율,자기자본구성비율,타인자본구성비율,자기자본배율,비유동비율,비유동장기적합률,유동비율,당좌비율,현금비율,매출채권비율,재고자산 대 순운전자본비율,매출채권 대 매입채무비율,매입채무 대 재고자산비율,부채비율,유동부채비율,단기차입금 대 총차입금비율,비유동부채비율,비유동부채 대 순운전자본비율,순운전자본비율,차입금의존도,차입금비율,유보액대비율,유보액 대 납입자본배율,유동자산집중도,비유동자산집중도,투자집중도,CASH FLOW 대 부채비율,CASH FLOW 대 차입금비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,정상영업이익대비이자보상배율,총자본회전률,경영자본회전률,자기자본회전률,자본금회전률,타인자본회전률,매입채무회전률,매입채무회전기간,유동자산회전률,당좌자산회전률,재고자산회전률,재고자산회전기간,재공품회전률,매출채권회전률,매출채권회전기간,비유동자산회전률,유형자산회전율,순운전자본회전률,운전자본회전률,1회전기간,부가가치(백만원),총자본투자효율,설비투자효율,기계투자효율,부가가치율,노동소득분배율,자본분배율,이윤분배율,영업활동으로 인한 현금흐름(간접법),투자활동으로 인한 현금흐름,재무활동으로 인한 현금흐름,유동부채,자산_x,자본,매출액(수익),매출액증가율,유형자산증가율,비유동생물자산증가율,투자부동산증가율,유동자산증가율,비유동자산증가율,재고자산증가율,정상영업이익증가율,순이익증가율,총포괄이익증가율,부실,총자본증가율,자기자본증가율
count,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4.654000e+03,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4.654000e+03,4654.000,4654.000,4.654000e+03,4654.000,4654.000,4654.000,4654.000,4.654000e+03,4.654000e+03,4.654000e+03,4.654000e+03,4.654000e+03,4.654000e+03,4.654000e+03,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000,4654.000
mean,46399.746,21.052,-0.639,-5.749,3.106,2.635,-0.024,4.443,-26.994,3.262,-0.193,223.834,136.763,73.554,100.940,-5.280,-1.671,1.653,-0.018,1.149,32.159,0.687,102.745,14.132,0.859,0.226,1.008,4.259,14.453,76.930,0.467,12.837,12.358,1939.129,79.881,73272.221,1776.937,3929.771,3396.973,46.483,20.629,134.605,37.076,53.336,52.808,47.325,1950.645,129.082,85.185,199.520,165.841,43.462,7983.878,155.289,2766.095,2657.950,141.874,100.050,47.695,40.863,246.222,12.401,23.840,76.408,44.180,24.321,77.232,101.234,138.853,11.703,912.598,3.401,1.531,9.961,0.924,1.140,2.154,40.931,2.360,572.506,33.870,2.175,2.972,417.711,45.303,7.787233e+03,10.690,63.036,2.265,13.210,1.171,0.573,0.044,3.086556e+05,17.196,234.993,1.174242e+07,16.984,103.369,-13.274,-66.101,1.069380e+08,-1.147158e+08,1.211016e+07,6.391855e+08,2.104710e+09,8.467170e+08,1.571906e+09,0.203,0.778,0.007,1.137,0.145,0.162,7.455,-0.700,-0.605,-0.562,0.013,0.105,0.107
std,48978.060,18.668,41.906,92.757,8.564,8.537,14.608,20.779,1483.701,10.715,20.270,1079.547,860.484,25.134,42.173,43.502,13.774,9.450,1.465,3.171,321.255,14.133,46.754,11.363,12.010,0.734,3.472,5.493,10.075,71.655,4.353,171.688,236.381,5127.621,105.230,185514.366,9937.830,13949.217,9945.232,19.261,18.068,210.888,18.925,19.250,20.581,21.046,4761.462,176.936,41.146,268.525,260.420,98.501,144043.887,2393.576,84969.439,67776.713,334.951,251.482,35.020,106.538,2715.761,23.160,18.357,211.895,25.290,109.766,140.665,167.009,212.201,38.216,20281.791,8.364,40.944,251.651,0.581,0.689,2.077,195.664,1.741,21369.204,48.633,1.339,2.130,7150.645,60.892,2.093593e+05,22.325,71.339,2.488,89.858,0.607,18.279,0.247,1.293460e+06,22.463,1585.133,2.500764e+08,62.594,1755.663,1755.334,2864.117,6.458446e+08,6.

주력산업과 동일

##### 피처의 밸류값이 대부분이 0인 컬럼 확인 및 제거
  - R & D 투자효율, 세금과공과 대 총비용비율, 비유동생물자산증가율 제거

In [ ]:
minor3[['R & D 투자효율']].value_counts() # 비주력산업 4654 중 0값이 3941로 제거

R & D 투자효율
0.00          3941
0.02            13
0.04            13
0.03            11
0.01            10
              ... 
2.71             1
2.72             1
2.75             1
2.83             1
543.87           1
Length: 342, dtype: int64

In [ ]:
minor3[['세금과공과 대 총비용비율']].value_counts() # 비주력산업 4654 중 0값이 3567개로 제거

세금과공과 대 총비용비율
0.00             3567
0.44               21
0.32               17
0.31               15
0.21               15
                 ... 
2.25                1
2.21                1
2.20                1
1.18                1
9.10                1
Length: 264, dtype: int64

In [ ]:
minor3[['비유동생물자산증가율']].value_counts() # 비주력산업 4654 중 0값이 4536으로 제거

비유동생물자산증가율
 0.000000     4536
-1.000000       10
 0.142102        1
 0.234251        1
 0.222767        1
              ... 
-0.024241        1
-0.024873        1
-0.025162        1
-0.033529        1
 26.614641       1
Length: 110, dtype: int64

In [ ]:
# R & D 투자효율, 세금과공과 대 총비용비율, 비유동생물자산증가율 제거
minor3.drop(['R & D 투자효율', '세금과공과 대 총비용비율', '비유동생물자산증가율'],axis=1, inplace=True)

#### 5. 단위 맞추기
  - 천원: '영업활동으로 인한 현금흐름(간접법)', '투자활동으로 인한 현금흐름',  '재무활동으로 인한 현금흐름', '유동부채','자산', '자본', '매출액(수익)'
  - 백만원: '부가가치(백만원)'

In [131]:
major3.columns.tolist()

['회사명',
 '거래소코드',
 '회계년도',
 '매출액총이익률',
 '매출액정상영업이익률',
 '매출액순이익률',
 '총자본사업이익률',
 '총자본정상영업이익률',
 '총자본순이익률',
 '자기자본정상영업이익률',
 '자기자본순이익률',
 '경영자본정상영업이익률',
 '경영자본순이익률',
 '자본금정상영업이익률',
 '자본금순이익률',
 '매출원가 대 매출액비율',
 '영업비용 대 영업수익비율',
 '기타손익비률',
 '금융손익비율',
 '금융비용부담률',
 '외환이익 대 매출액비율',
 '광고선전비 대 매출액비율',
 '세금과공과 대 세금과공과 차감전순이익률',
 '기업순이익률',
 '수지비율(관계기업투자손익 제외)',
 '인건비 대 총비용비율',
 '금융비용 대 총비용비율',
 '감가상각비 대 총비용비율',
 '감가상각률',
 '누적감가상각률',
 '이자부담률',
 '지급이자율',
 '차입금평균이자율',
 '유보율',
 '적립금비율(재정비율)',
 '1주당매출액',
 '1주당순이익',
 '1주당 CASH FLOW',
 '1주당정상영업이익',
 '유동자산구성비율',
 '재고자산 대 유동자산비율',
 '유동자산 대 비유동자산비율',
 '당좌자산구성비율',
 '비유동자산구성비율',
 '자기자본구성비율',
 '타인자본구성비율',
 '자기자본배율',
 '비유동비율',
 '비유동장기적합률',
 '유동비율',
 '당좌비율',
 '현금비율',
 '매출채권비율',
 '재고자산 대 순운전자본비율',
 '매출채권 대 매입채무비율',
 '매입채무 대 재고자산비율',
 '부채비율',
 '유동부채비율',
 '단기차입금 대 총차입금비율',
 '비유동부채비율',
 '비유동부채 대 순운전자본비율',
 '순운전자본비율',
 '차입금의존도',
 '차입금비율',
 '유보액대비율',
 '유보액 대 납입자본배율',
 '유동자산집중도',
 '비유동자산집중도',
 '투자집중도',
 'CASH FLOW 대 부채비율',
 'CASH FLOW 대 차입금비율',
 'CASH FLOW 대 총자본비율',


In [132]:
minor3.columns.tolist()

['회사명',
 '거래소코드',
 '회계년도',
 '매출액총이익률',
 '매출액정상영업이익률',
 '매출액순이익률',
 '총자본사업이익률',
 '총자본정상영업이익률',
 '총자본순이익률',
 '자기자본정상영업이익률',
 '자기자본순이익률',
 '경영자본정상영업이익률',
 '경영자본순이익률',
 '자본금정상영업이익률',
 '자본금순이익률',
 '매출원가 대 매출액비율',
 '영업비용 대 영업수익비율',
 '기타손익비률',
 '금융손익비율',
 '금융비용부담률',
 '외환이익 대 매출액비율',
 '광고선전비 대 매출액비율',
 '세금과공과 대 세금과공과 차감전순이익률',
 '기업순이익률',
 '수지비율(관계기업투자손익 제외)',
 '인건비 대 총비용비율',
 '금융비용 대 총비용비율',
 '감가상각비 대 총비용비율',
 '감가상각률',
 '누적감가상각률',
 '이자부담률',
 '지급이자율',
 '차입금평균이자율',
 '유보율',
 '적립금비율(재정비율)',
 '1주당매출액',
 '1주당순이익',
 '1주당 CASH FLOW',
 '1주당정상영업이익',
 '유동자산구성비율',
 '재고자산 대 유동자산비율',
 '유동자산 대 비유동자산비율',
 '당좌자산구성비율',
 '비유동자산구성비율',
 '자기자본구성비율',
 '타인자본구성비율',
 '자기자본배율',
 '비유동비율',
 '비유동장기적합률',
 '유동비율',
 '당좌비율',
 '현금비율',
 '매출채권비율',
 '재고자산 대 순운전자본비율',
 '매출채권 대 매입채무비율',
 '매입채무 대 재고자산비율',
 '부채비율',
 '유동부채비율',
 '단기차입금 대 총차입금비율',
 '비유동부채비율',
 '비유동부채 대 순운전자본비율',
 '순운전자본비율',
 '차입금의존도',
 '차입금비율',
 '유보액대비율',
 '유보액 대 납입자본배율',
 '유동자산집중도',
 '비유동자산집중도',
 '투자집중도',
 'CASH FLOW 대 부채비율',
 'CASH FLOW 대 차입금비율',
 'CASH FLOW 대 총자본비율',


In [133]:
# 백만원에서 원으로 변환하는 함수
def to_won(value):
    return value * 1000000

In [134]:
# 백만원을 원으로 변환
target_columns = ['부가가치(백만원)']

for column in target_columns:
    major3[column] = major3[column].apply(to_won)
    minor3[column] = minor3[column].apply(to_won)


In [135]:
# 천원에서 원으로 변환하는 함수
def to_won2(value):
    return value * 1000

In [136]:
# 천원을 원으로 변환
target_columns2 = ['영업활동으로 인한 현금흐름(간접법)','투자활동으로 인한 현금흐름', '재무활동으로 인한 현금흐름','유동부채','자산_x','자본','매출액(수익)']

for column in target_columns2:
    major3[column] = major3[column].apply(to_won2)
    minor3[column] = minor3[column].apply(to_won2)

#### 6. 부실기업과 부실이 아닌 기업 비교
  - major3: (6507, 125)
  - minor3: (4654, 125)

주력산업

In [137]:
major3[major3['부실']==1].describe()

,거래소코드,매출액총이익률,매출액정상영업이익률,매출액순이익률,총자본사업이익률,총자본정상영업이익률,총자본순이익률,자기자본정상영업이익률,자기자본순이익률,경영자본정상영업이익률,경영자본순이익률,자본금정상영업이익률,자본금순이익률,매출원가 대 매출액비율,영업비용 대 영업수익비율,기타손익비률,금융손익비율,금융비용부담률,외환이익 대 매출액비율,광고선전비 대 매출액비율,세금과공과 대 세금과공과 차감전순이익률,기업순이익률,수지비율(관계기업투자손익 제외),인건비 대 총비용비율,금융비용 대 총비용비율,감가상각비 대 총비용비율,감가상각률,누적감가상각률,이자부담률,지급이자율,차입금평균이자율,유보율,적립금비율(재정비율),1주당매출액,1주당순이익,1주당 CASH FLOW,1주당정상영업이익,유동자산구성비율,재고자산 대 유동자산비율,유동자산 대 비유동자산비율,당좌자산구성비율,비유동자산구성비율,자기자본구성비율,타인자본구성비율,자기자본배율,비유동비율,비유동장기적합률,유동비율,당좌비율,현금비율,매출채권비율,재고자산 대 순운전자본비율,매출채권 대 매입채무비율,매입채무 대 재고자산비율,부채비율,유동부채비율,단기차입금 대 총차입금비율,비유동부채비율,비유동부채 대 순운전자본비율,순운전자본비율,차입금의존도,차입금비율,유보액대비율,유보액 대 납입자본배율,유동자산집중도,비유동자산집중도,투자집중도,CASH FLOW 대 부채비율,CASH FLOW 대 차입금비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,정상영업이익대비이자보상배율,총자본회전률,경영자본회전률,자기자본회전률,자본금회전률,타인자본회전률,매입채무회전률,매입채무회전기간,유동자산회전률,당좌자산회전률,재고자산회전률,재고자산회전기간,재공품회전률,매출채권회전률,매출채권회전기간,비유동자산회전률,유형자산회전율,순운전자본회전률,운전자본회전률,1회전기간,부가가치(백만원),총자본투자효율,설비투자효율,기계투자효율,부가가치율,노동소득분배율,자본분배율,이윤분배율,영업활동으로 인한 현금흐름(간접법),투자활동으로 인한 현금흐름,재무활동으로 인한 현금흐름,유동부채,자산_x,자본,매출액(수익),매출액증가율,유형자산증가율,투자부동산증가율,유동자산증가율,비유동자산증가율,재고자산증가율,정상영업이익증가율,순이익증가율,총포괄이익증가율,부실,총자본증가율,자기자본증가율
count,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,1.070000e+02,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,1.070000e+02,1.070000e+02,1.070000e+02,1.070000e+02,1.070000e+02,1.070000e+02,1.070000e+02,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.0,107.000000,107.000000
mean,66732.149533,4.525234,-39.269720,-107.166168,-12.579346,-13.042430,-31.147290,-40.336168,-107.351121,-17.814299,-41.314019,-111.667664,-210.148692,93.605607,139.654579,-59.202336,-11.569720,9.360374,0.278411,0.598972,2.109907,-28.361869,175.608224,12.651589,3.725514,4.693084,13.020467,85.631495,3.727757,9.862336,10.472056,135.492710,-71.377103,5248.775888,-2698.233645,-410.885140,-868.915047,46.720093,17.112336,133.874112,39.038411,53.279907,28.462336,75.422243,208.939159,328.214673,214.904112,111.110000,94.736168,17.857383,422.769907,29.212617,224.069813,254.596262,538.974766,474.936449,58.751121,64.038505,31.266449,-20.223178,47.600654,314.037944,3.596729,1.994112,152.983178,162.796449,203.470187,-12.572150,-27.627757,-9.384673,-33.804019,-10.186542,0.578785,0.704766,2.132523,7.870561,1.098037,8.978879,83.665140,1.402056,1.824486,22.816542,78.029907,282.216916,7.029907,116.013551,1.321028,5.583925,0.564486,0.390935,0.123458,-2.406292e+10,-19.994766,-474.161682,-3466.478692,-52.475701,207.992430,-157.525140,-266.506636,-1.775400e+10,-2.135085e+09,1.232730e+10,2.322422e+11,2.963767e+11,4.583655e+09,1.611351e+11,-0.134499,0.699210,-0.006992,-0.105497,0.192458,0.779407,-1.719009,-3.041565,-1.776671,1.0,-0.129778,-0.635902
std,43116.479886,36.656557,62.674295,200.704372,14.558244,14.556937,35.795716,56.887105,179.162893,30.689237,54.448714,191.561955,299.487835,38.881607,62.738844,129.384742,25.953984,20.211779,5.481731,1.335390,19.155209,35.431631,139.620885,7.137726,3.993771,4.765801,7.297162,99.373314,4.836482,12.172014,23.509852,476.828743,60

In [138]:
major3[major3['부실']==0].describe()

,거래소코드,매출액총이익률,매출액정상영업이익률,매출액순이익률,총자본사업이익률,총자본정상영업이익률,총자본순이익률,자기자본정상영업이익률,자기자본순이익률,경영자본정상영업이익률,경영자본순이익률,자본금정상영업이익률,자본금순이익률,매출원가 대 매출액비율,영업비용 대 영업수익비율,기타손익비률,금융손익비율,금융비용부담률,외환이익 대 매출액비율,광고선전비 대 매출액비율,세금과공과 대 세금과공과 차감전순이익률,기업순이익률,수지비율(관계기업투자손익 제외),인건비 대 총비용비율,금융비용 대 총비용비율,감가상각비 대 총비용비율,감가상각률,누적감가상각률,이자부담률,지급이자율,차입금평균이자율,유보율,적립금비율(재정비율),1주당매출액,1주당순이익,1주당 CASH FLOW,1주당정상영업이익,유동자산구성비율,재고자산 대 유동자산비율,유동자산 대 비유동자산비율,당좌자산구성비율,비유동자산구성비율,자기자본구성비율,타인자본구성비율,자기자본배율,비유동비율,비유동장기적합률,유동비율,당좌비율,현금비율,매출채권비율,재고자산 대 순운전자본비율,매출채권 대 매입채무비율,매입채무 대 재고자산비율,부채비율,유동부채비율,단기차입금 대 총차입금비율,비유동부채비율,비유동부채 대 순운전자본비율,순운전자본비율,차입금의존도,차입금비율,유보액대비율,유보액 대 납입자본배율,유동자산집중도,비유동자산집중도,투자집중도,CASH FLOW 대 부채비율,CASH FLOW 대 차입금비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,정상영업이익대비이자보상배율,총자본회전률,경영자본회전률,자기자본회전률,자본금회전률,타인자본회전률,매입채무회전률,매입채무회전기간,유동자산회전률,당좌자산회전률,재고자산회전률,재고자산회전기간,재공품회전률,매출채권회전률,매출채권회전기간,비유동자산회전률,유형자산회전율,순운전자본회전률,운전자본회전률,1회전기간,부가가치(백만원),총자본투자효율,설비투자효율,기계투자효율,부가가치율,노동소득분배율,자본분배율,이윤분배율,영업활동으로 인한 현금흐름(간접법),투자활동으로 인한 현금흐름,재무활동으로 인한 현금흐름,유동부채,자산_x,자본,매출액(수익),매출액증가율,유형자산증가율,투자부동산증가율,유동자산증가율,비유동자산증가율,재고자산증가율,정상영업이익증가율,순이익증가율,총포괄이익증가율,부실,총자본증가율,자기자본증가율
count,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6.400000e+03,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6.400000e+03,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6.400000e+03,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.00000,6400.000000,6400.000000,6400.000000,6.400000e+03,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6.400000e+03,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6.400000e+03,6400.000000,6400.000000,6.400000e+03,6400.000000,6400.000000,6400.000000,6400.000000,6.400000e+03,6.400000e+03,6.400000e+03,6.400000e+03,6.400000e+03,6.400000e+03,6.400000e+03,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.0,6400.000000,6400.000000
mean,61720.106250,21.291472,1.344675,-2.533425,3.892355,3.469809,1.040394,5.531381,-1.765173,3.944734,0.963622,151.107531,93.132992,76.364778,98.862141,-2.708527,-1.266155,0.957637,-0.009378,0.809208,14.973345,1.704822,99.490598,15.264602,0.750194,4.277670,13.617622,82.804544,0.140105,7.252981,5.665095,1540.208495,79.537705,4.031342e+04,1561.555363,3125.940806,2382.905966,51.138569,24.564370,144.048298,39.144042,48.613356,56.056059,43.973002,1585.504423,124.458067,80.062967,251.188259,202.167841,54.443622,1.452407e+03,295.401809,416.028459,735.396805,141.855959,99.155728,51.177633,40.404644,5.443793e+02,18.010106,23.153864,81.345548,48.923484,16.533963,74.91660,77.202953,128.050073,17.442011,1.256749e+03,4.428925,3.948820,20.460448,0.920305,1.063367,2.014055,26.273514,2.650748,24.588797,35.173195,1.887580,2.671113,85.093403,55.492413,1.508926e+03,7.464644,72.509736,2.304159,6.815091,1.104372,1.536712,0.022631,3.475339e+11,17.955705,155.148359,2.040223e+05,20.819881,91.685627,1.783123,-35.109573,1.548132e+11,-1.472696e+11,1.147089e+09,5.122776e+11,1.903023e+12,9.570029e+11,1.560319e+12,0.094053,0.657689,0.343273,0.094998,0.143784,0.441349,-0.306451,-5.539061,-0.447723,0.0,0.087338,0.121867
std,54889.020427,17.690179,59.720535,64.204777,8.221078,8.195906,11.822006,20.565716,60.374779,10.468700,17.275943,399.081446,356.043831,21.024763,59.753877,24.064713,13.896201,5.022397,1.247907,2.897091,477.825380,11.325473,28.693332,9.647280,2.17815

비주력산업

In [139]:
minor3[minor3['부실']==1].describe()

,거래소코드,매출액총이익률,매출액정상영업이익률,매출액순이익률,총자본사업이익률,총자본정상영업이익률,총자본순이익률,자기자본정상영업이익률,자기자본순이익률,경영자본정상영업이익률,경영자본순이익률,자본금정상영업이익률,자본금순이익률,매출원가 대 매출액비율,영업비용 대 영업수익비율,기타손익비률,금융손익비율,금융비용부담률,외환이익 대 매출액비율,광고선전비 대 매출액비율,세금과공과 대 세금과공과 차감전순이익률,기업순이익률,수지비율(관계기업투자손익 제외),인건비 대 총비용비율,금융비용 대 총비용비율,감가상각비 대 총비용비율,감가상각률,누적감가상각률,이자부담률,지급이자율,차입금평균이자율,유보율,적립금비율(재정비율),1주당매출액,1주당순이익,1주당 CASH FLOW,1주당정상영업이익,유동자산구성비율,재고자산 대 유동자산비율,유동자산 대 비유동자산비율,당좌자산구성비율,비유동자산구성비율,자기자본구성비율,타인자본구성비율,자기자본배율,비유동비율,비유동장기적합률,유동비율,당좌비율,현금비율,매출채권비율,재고자산 대 순운전자본비율,매출채권 대 매입채무비율,매입채무 대 재고자산비율,부채비율,유동부채비율,단기차입금 대 총차입금비율,비유동부채비율,비유동부채 대 순운전자본비율,순운전자본비율,차입금의존도,차입금비율,유보액대비율,유보액 대 납입자본배율,유동자산집중도,비유동자산집중도,투자집중도,CASH FLOW 대 부채비율,CASH FLOW 대 차입금비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,정상영업이익대비이자보상배율,총자본회전률,경영자본회전률,자기자본회전률,자본금회전률,타인자본회전률,매입채무회전률,매입채무회전기간,유동자산회전률,당좌자산회전률,재고자산회전률,재고자산회전기간,재공품회전률,매출채권회전률,매출채권회전기간,비유동자산회전률,유형자산회전율,순운전자본회전률,운전자본회전률,1회전기간,부가가치(백만원),총자본투자효율,설비투자효율,기계투자효율,부가가치율,노동소득분배율,자본분배율,이윤분배율,영업활동으로 인한 현금흐름(간접법),투자활동으로 인한 현금흐름,재무활동으로 인한 현금흐름,유동부채,자산_x,자본,매출액(수익),매출액증가율,유형자산증가율,투자부동산증가율,유동자산증가율,비유동자산증가율,재고자산증가율,정상영업이익증가율,순이익증가율,총포괄이익증가율,부실,총자본증가율,자기자본증가율
count,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,5.900000e+01,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,5.900000e+01,5.900000e+01,5.900000e+01,5.900000e+01,5.900000e+01,5.900000e+01,5.900000e+01,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.0,59.000000,59.000000
mean,49027.966102,13.612712,-49.493559,-113.310000,-13.270508,-13.934915,-31.573559,-27.054915,-1882.730339,-18.243559,-42.846271,-84.699153,-182.606780,86.387288,152.261017,-60.978814,-7.224237,7.140678,0.768136,1.457627,3.646271,-28.970508,195.357119,10.635424,3.332373,2.707119,15.406949,76.354407,3.864746,146.627288,28.549492,34.052542,4.783051,11341.713729,-3975.154237,241.848475,-1242.037627,47.601017,14.287288,117.497966,40.650000,52.398983,32.244746,72.972034,120.530508,184.002203,99.805085,139.464237,123.300678,24.463898,4364.643390,141.806780,340.681525,2558.593051,262.330508,190.286102,47.662712,72.044576,191.691356,-9.209153,38.602712,138.397797,1.736441,0.901525,131.529661,152.722542,182.169153,-13.490847,-241.300508,-5.535593,-20.882881,-12.347458,0.563220,0.768136,1.609322,4.726102,0.917288,10.649322,79.037458,1.204237,1.473729,192.807627,78.988644,270.778644,25.898305,118.616102,1.268475,17.283220,0.565763,0.468983,0.032712,-1.755921e+10,-19.527627,-374.170847,14879.929492,-71.687797,296.032034,-267.218475,-583.207288,-1.137703e+09,3.953258e+09,-3.343266e+09,1.666426e+11,2.540812e+11,3.271652e+09,1.952448e+11,-0.024302,0.115495,0.014881,-0.076764,-0.091158,0.029809,-1.809982,-4.569062,-12.816812,1.0,-0.103902,0.307953
std,51093.342193,31.741890,140.590631,261.703506,16.442659,16.455760,27.321269,45.840139,13146.544984,19.716613,35.120925,150.479760,224.270962,31.741890,140.643396,144.233717,10.837350,9.684935,6.756234,3.291620,19.677283,27.161642,257.676509,9.816468,2.628270,3.186229,9.374618,105.988558,2.783653,779.833871,103.037261,296.821466,161.194159,21587.445191,9237.290535,3942.467327,3115.427904,16.277475,18.065263,109.397346,16.0989

In [140]:
major3[major3['부실']==0].describe()

,거래소코드,매출액총이익률,매출액정상영업이익률,매출액순이익률,총자본사업이익률,총자본정상영업이익률,총자본순이익률,자기자본정상영업이익률,자기자본순이익률,경영자본정상영업이익률,경영자본순이익률,자본금정상영업이익률,자본금순이익률,매출원가 대 매출액비율,영업비용 대 영업수익비율,기타손익비률,금융손익비율,금융비용부담률,외환이익 대 매출액비율,광고선전비 대 매출액비율,세금과공과 대 세금과공과 차감전순이익률,기업순이익률,수지비율(관계기업투자손익 제외),인건비 대 총비용비율,금융비용 대 총비용비율,감가상각비 대 총비용비율,감가상각률,누적감가상각률,이자부담률,지급이자율,차입금평균이자율,유보율,적립금비율(재정비율),1주당매출액,1주당순이익,1주당 CASH FLOW,1주당정상영업이익,유동자산구성비율,재고자산 대 유동자산비율,유동자산 대 비유동자산비율,당좌자산구성비율,비유동자산구성비율,자기자본구성비율,타인자본구성비율,자기자본배율,비유동비율,비유동장기적합률,유동비율,당좌비율,현금비율,매출채권비율,재고자산 대 순운전자본비율,매출채권 대 매입채무비율,매입채무 대 재고자산비율,부채비율,유동부채비율,단기차입금 대 총차입금비율,비유동부채비율,비유동부채 대 순운전자본비율,순운전자본비율,차입금의존도,차입금비율,유보액대비율,유보액 대 납입자본배율,유동자산집중도,비유동자산집중도,투자집중도,CASH FLOW 대 부채비율,CASH FLOW 대 차입금비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,정상영업이익대비이자보상배율,총자본회전률,경영자본회전률,자기자본회전률,자본금회전률,타인자본회전률,매입채무회전률,매입채무회전기간,유동자산회전률,당좌자산회전률,재고자산회전률,재고자산회전기간,재공품회전률,매출채권회전률,매출채권회전기간,비유동자산회전률,유형자산회전율,순운전자본회전률,운전자본회전률,1회전기간,부가가치(백만원),총자본투자효율,설비투자효율,기계투자효율,부가가치율,노동소득분배율,자본분배율,이윤분배율,영업활동으로 인한 현금흐름(간접법),투자활동으로 인한 현금흐름,재무활동으로 인한 현금흐름,유동부채,자산_x,자본,매출액(수익),매출액증가율,유형자산증가율,투자부동산증가율,유동자산증가율,비유동자산증가율,재고자산증가율,정상영업이익증가율,순이익증가율,총포괄이익증가율,부실,총자본증가율,자기자본증가율
count,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6.400000e+03,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6.400000e+03,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6.400000e+03,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.00000,6400.000000,6400.000000,6400.000000,6.400000e+03,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6.400000e+03,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6.400000e+03,6400.000000,6400.000000,6.400000e+03,6400.000000,6400.000000,6400.000000,6400.000000,6.400000e+03,6.400000e+03,6.400000e+03,6.400000e+03,6.400000e+03,6.400000e+03,6.400000e+03,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.000000,6400.0,6400.000000,6400.000000
mean,61720.106250,21.291472,1.344675,-2.533425,3.892355,3.469809,1.040394,5.531381,-1.765173,3.944734,0.963622,151.107531,93.132992,76.364778,98.862141,-2.708527,-1.266155,0.957637,-0.009378,0.809208,14.973345,1.704822,99.490598,15.264602,0.750194,4.277670,13.617622,82.804544,0.140105,7.252981,5.665095,1540.208495,79.537705,4.031342e+04,1561.555363,3125.940806,2382.905966,51.138569,24.564370,144.048298,39.144042,48.613356,56.056059,43.973002,1585.504423,124.458067,80.062967,251.188259,202.167841,54.443622,1.452407e+03,295.401809,416.028459,735.396805,141.855959,99.155728,51.177633,40.404644,5.443793e+02,18.010106,23.153864,81.345548,48.923484,16.533963,74.91660,77.202953,128.050073,17.442011,1.256749e+03,4.428925,3.948820,20.460448,0.920305,1.063367,2.014055,26.273514,2.650748,24.588797,35.173195,1.887580,2.671113,85.093403,55.492413,1.508926e+03,7.464644,72.509736,2.304159,6.815091,1.104372,1.536712,0.022631,3.475339e+11,17.955705,155.148359,2.040223e+05,20.819881,91.685627,1.783123,-35.109573,1.548132e+11,-1.472696e+11,1.147089e+09,5.122776e+11,1.903023e+12,9.570029e+11,1.560319e+12,0.094053,0.657689,0.343273,0.094998,0.143784,0.441349,-0.306451,-5.539061,-0.447723,0.0,0.087338,0.121867
std,54889.020427,17.690179,59.720535,64.204777,8.221078,8.195906,11.822006,20.565716,60.374779,10.468700,17.275943,399.081446,356.043831,21.024763,59.753877,24.064713,13.896201,5.022397,1.247907,2.897091,477.825380,11.325473,28.693332,9.647280,2.17815

In [141]:
major3.to_csv('./drive/MyDrive/공유문서함/상장csv/240316상장_주력전처리완료.csv')
minor3.to_csv('./drive/MyDrive/공유문서함/상장csv/240316상장_비주력전처리완료.csv')

In [142]:
display(major3.shape)
display(minor3.shape)

(6507, 125)

(4654, 125)